<a href="https://colab.research.google.com/github/Mikler87/dbg-pds/blob/master/FP_GAN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation and authentication

In [1]:
import os, sys, math
import numpy as np
import scipy as sp
import scipy.stats
import time
import datetime
import string
import random
from matplotlib import pyplot as plt
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.0.0
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
import pickle
#AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

import torch
import torchvision
import sys
#import tensorboardcolab
#!pip install tensorboard==2.2.2
from tensorboard import version

print('__Python VERSION:', sys.version)
print('__Tensorflow VERSION',tf.__version__)
print('__Tensorboard VERSION:', version.VERSION)
print('__pyTorch VERSION:', torch.__version__)
print('__Torchvision VERSION',torchvision.__version__)
print('__CUDA VERSION',torch.version.cuda )
from subprocess import call
!nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
if torch.cuda.device_count() >0:
  print('Active CUDA Device: GPU', torch.cuda.current_device())
  print('Current device name',torch.cuda.get_device_name(0))
else:
  print('No Cuda Device')



print('Mounting google drive...')
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/My Drive"
!mkdir FinalProject-GCP
%cd "/content/drive/My Drive/FinalProject-GCP"

#  we authenticate with the GCS to enable access to Dataproc and AI-Platform.
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
#%pwd
#%ls

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.0.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
Tensorflow version 2.3.0
__Python VERSION: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
__Tensorflow VERSION 2.3.0
__Tensorboard VERSION: 2.3.0
__pyTorch VERSION: 1.6.0+cu101
__Torchvision VERSION 0.7.0+cu101
__CUDA VERSION 10.1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
__CUDNN VERSION: 7603
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Current device name Tesla P100-PCIE-16GB
Mounting google drive...
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.

# Project creation GCP

In [2]:
PROJECT = 'fpcityuni-2020' ### USE YOUR PROJECT ID HERE! ###
!gcloud config set project $PROJECT
REGION = 'us-central1'
CLUSTER = '{}-cluster'.format(PROJECT)
!gcloud config set compute/region $REGION
!gcloud config set dataproc/region $REGION

!gcloud config list # show some information

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Updated property [compute/region].
Updated property [dataproc/region].
[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
region = us-central1
[core]
account = 87mazar@gmail.com
project = fpcityuni-2020
[dataproc]
region = us-central1

Your active configuration is: [default]


In [3]:
BUCKET = 'gs://{}-storage'.format(PROJECT)
!gsutil mb $BUCKET

Creating gs://fpcityuni-2020-storage/...
ServiceException: 409 Bucket fpcityuni-2020-storage already exists.


# Set local directory
Make sure the python file and dataset label sheet (txt) is saved there
The dataset is saved in the cloud for google drive cannot handle big size folders

In [4]:
%cd "/content/drive/My Drive/FinalProject-GCP/"
!ls -1

/content/drive/My Drive/FinalProject-GCP
AllFile.ipynb
brats_auc.py
brats_syn_256_lambda0.1
celeba_1
City_server_FP_GAN.ipynb
CodeCombined.py
CodeCombined_tf_1x_CPU.py
CodeCombined_tf_1x_GPU.py
CodeCombined_tf_1x.py
CodeCombined_tf_2x_CPU.py
CodeCombined_tf_2x_GPU.py
'Copy of FP-GAN.ipynb'
data
dataset__gcp.zip
dataset.zip
data.zip
dist
download.sh
Fixed-Point-GAN.ipynb
fpcityuni-2020-59e3d89958d7.json
FP-GAN.ipynb
FP-GAN-Org-v0.ipynb
FP-GAN-v1.ipynb
FP-GAN-v2.ipynb
FP-GAN-v3.ipynb
FP-GAN-v4.ipynb
GAN_MSG.py
GAN_nGPU_GCP_Method1_LastResolutionLoss_Separable_conv_additional_layer.py
GAN_nGPU_GCP_Method1_LastResolutionLoss_Separable_conv.py
GAN_nGPU_GCP_Method1_LastResolutionLoss_v2.py
GAN_nGPU_GCP_Method2_scaledimagesaveraging.py
GAN_nGPU_GCP_Method2_scaledimagesaveraging_Separable_conv_additional_layer.py
GAN_nGPU_GCP_Method2_scaledimagesaveraging_Separable_conv.py
GAN_nGPU_GCP_Method2_scaledimagesaveraging_v2.py
GAN_nGPU_GCP_Org_1024.py
GAN_tf_2x_nGPU__New.py
GAN_tf_2x_nGPU.py
GAN_tf_

# Generator Discriminator: Smaller Bottleneck 64x64x256
There are 2 versions of the loss: One where the loss is considered the loss of the last 1024x1024 image while method 2 is the avarage loss of multiscaled images

There are 4 differents versions of Generator Discriminator. Each One is saved in different files. 

#Method 1: 

In [5]:
%%writefile GAN_nGPU_GCP_Method1_LastResolutionLoss_v2.py

# inserting a file with huge number of images doesn't work with google drive, gcp is needed to read the data from there
# Model
# works with 2 gpu not 3 and with 20 batch size 10.30 mins
# Change the code to blocks

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.nn import ModuleList, Conv2d


class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout, kernels_per_layer=1, K_depth=3, S_depth=1, P_depth=1, D_depth=1, K_point=1, S_point=1, P_point=0, D_point=1, bias=False):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=K_depth,stride=S_depth, padding=P_depth,dilation=D_depth, groups=nin,bias=bias)
        self.pointwise = nn.Conv2d(nin * kernels_per_layer, nout, kernel_size=K_point,stride=S_point,padding=P_point,dilation=D_point, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class ResidualBlock(nn.Module):
    """Residual Block with instance normalization."""
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            depthwise_separable_conv(nin=dim_in,nout=dim_out, kernels_per_layer=1,K_depth=3,S_depth=1,P_depth=1,D_depth=1,K_point=1,S_point=1,P_point=0,D_point=1,bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            depthwise_separable_conv(nin=dim_in,nout=dim_out, kernels_per_layer=1,K_depth=3,S_depth=1,P_depth=1,D_depth=1,K_point=1,S_point=1,P_point=0,D_point=1,bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)


class Generator(nn.Module):
    """Generator network."""
    def __init__(self, ngpu=0, conv_dim=16, c_dim=1, repeat_num=6,depth=4):
        super(Generator, self).__init__()
        self.ngpu = torch.cuda.device_count()
        
        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True))

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(depth):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))
        
        self.main_DownSampling_bottleneck = nn.Sequential(*layers)
        
        # create a module list of the other required general convolution blocks
        self.UpsamplingLayers = ModuleList()
        self.rgb_converters = ModuleList()

        # Up-sampling layers.
        def conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential( nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=K, stride=S, padding=P, bias=bias_),
            nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True))
        
        # to rgb conversion
        def to_rgb(in_channels):
          layers_rgb = []
          layers_rgb.append(nn.Sequential(Conv2d(in_channels, 3, (1, 1),(1,1),(0,0), bias=True)))
          #layers_rgb.append(nn.Tanh())
          out = nn.Sequential(*layers_rgb)
          return out

        for i in range(depth):
          if i != (depth-1):
            self.UpsamplingLayers.append(conv_block(curr_dim,4,2,1,False))
            rgb = to_rgb(int(curr_dim//2))
            self.rgb_converters.append(rgb)
            curr_dim = curr_dim // 2
          else :
            self.UpsamplingLayers.append(conv_block(curr_dim,4,2,1,False)) # curr_dim,7,1,3,False)
            rgb = to_rgb(int(curr_dim//2))
            self.rgb_converters.append(rgb)
            curr_dim = curr_dim // 2
          

    def forward(self, x, c):
        # Replicate spatially and concatenate domain information.
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)

        #print(self.UpsamplingLayers)
        #print(self.rgb_converters)

        outputs=[]
        Y=self.main_DownSampling_bottleneck(x) # start the computational pipeline
        #counter=0
        #print("Y after downsmapling and bottleneck",Y.size())
        for block, converter in zip(self.UpsamplingLayers, self.rgb_converters):
            #print("Y again",Y.size())
            Y = block(Y)
            outputs.append(converter(Y))
            #counter=counter+1
            #print(counter)
        return outputs



# Discriminator code using blocks

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.nn import ModuleList, Conv2d


class Discriminator(nn.Module):
    """Discriminator network with PatchGAN."""
    def __init__(self, ngpu=0, image_size=1024, conv_dim=16, c_dim=5, repeat_num=8,depth=4):
        super(Discriminator, self).__init__()
        self.ngpu=torch.cuda.device_count()

        def from_rgb(out_channels):
          return Conv2d(3, out_channels, (1, 1), bias=True)

        curr_dim = conv_dim
        self.rgb_to_features = ModuleList()

  
        # Discrimintor Conv Block
        def dis_initial_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))

        def dis_mid_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim+3, curr_dim, kernel_size=3, stride=1, padding=1,bias=False), nn.LeakyReLU(0.01),
                               nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))
        
        #def dis_mid_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          #return nn.Sequential(nn.Conv2d(curr_dim+3, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               #nn.LeakyReLU(0.01))
          
        def dis_conv_block_without_rgb(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))

        # Initiate discrimator blocks module list
        self.Dis_blocks = ModuleList()

        # Fill the module list of both discriminator blocks and rgb converter
        for i in range(1, repeat_num):
          if i == 1: # first layer
            self.Dis_blocks.append(dis_initial_conv_block(curr_dim,4,2,1,False))
            from_rgb_ = from_rgb(int(curr_dim))
            self.rgb_to_features.append(from_rgb_)            
            curr_dim = curr_dim * 2
          elif i < (repeat_num-3) and (i != 1):
            self.Dis_blocks.append(dis_mid_conv_block(curr_dim,4,2,1,False))
            from_rgb_ = from_rgb(int(curr_dim))
            self.rgb_to_features.append(from_rgb_)            
            curr_dim = curr_dim * 2
          elif i == repeat_num: # last layer
            self.Dis_blocks.append(dis_conv_block_without_rgb(curr_dim,4,2,1,False))
          else:
            self.Dis_blocks.append(dis_conv_block_without_rgb(curr_dim,4,2,1,False))      
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num)) # this code was modified to fit kernel 8x8 to get the class
        print("kernel size",kernel_size)

        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        if image_size == 1024:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01),nn.Conv2d(c_dim, c_dim, kernel_size=kernel_size, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01),nn.Conv2d(c_dim, c_dim, kernel_size=kernel_size-2, stride=1, padding=0,bias=False), nn.LeakyReLU(0.01))
        elif image_size == 512:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size+2, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01)) 
        
        elif image_size == 256:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size+1, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01)) 

        
        
    def forward(self, inputs):
        #repeat_num=8;
        #depth=4

        # First Input
        y = self.rgb_to_features[0](inputs[0])
        #print("y rgb to features",y.size())
        y = self.Dis_blocks[0](y)
        #print("dis_block",y.size())

        #counter=0
          
        for x, block in \
                zip((inputs[1:]),
                    (self.Dis_blocks[1:])):
            
            #def get_shapes(Depth):
            #  return [inputs[i].size() for i in range(Depth)]
            #print('full input shape:', get_shapes(len(inputs)) )

            #print('x: ', x.size())
            #input_part = converter(x)  # convert the input:
            #print("input_part: ", input_part.size() )
            #print(converter)
            y = torch.cat((x, y), dim=1)  # concatenate the inputs:
            #print("concat input_part(here x instead) and y:",y.size())
            #print('block:', block)
            y = block(y)  # apply the block
            #print("y block(y):",y.size())
            #counter=counter+1
            #print(counter)

        y = self.Dis_blocks[-3](y)
        #print(y.size())
        y = self.Dis_blocks[-2](y)
        #print(y.size())
        y = self.Dis_blocks[-1](y)
        #print("y last",y.size())
        # calculate the final block:
        #input_part = self.final_converter(inputs[0])#
        #y = th.cat((input_part, y), dim=1)
        #y = self.final_block(y)

        out_src = self.conv1(y)
        #print("out_src: ",out_src.size())
        out_cls = self.conv2(y)
        #print("out_cls: ",out_cls.size())
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))


"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class ResidualBlock(nn.Module):
    """"""Residual Block with instance normalization.""""""
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True), # nn.ReLU(inplace=True) originally
            nn.Conv2d(dim_out, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)


class Generator(nn.Module):
    """"""Generator network.""""""
    def __init__(self, ngpu=0, conv_dim=64, c_dim=5, repeat_num=6):
        super(Generator, self).__init__()
        self.ngpu = torch.cuda.device_count()
        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True)) # before it was True

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(2):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True)) # before it was True
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))

        # Up-sampling layers.
        for i in range(2):
            layers.append(nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True)) # was True
            curr_dim = curr_dim // 2

        layers.append(nn.Conv2d(curr_dim, 3, kernel_size=7, stride=1, padding=3, bias=False))
        self.main = nn.Sequential(*layers)

    def forward(self, x, c):
        # Replicate spatially and concatenate domain information.
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)
        return self.main(x)


class Discriminator(nn.Module):
    """"""Discriminator network with PatchGAN.""""""
    def __init__(self, ngpu=0, image_size=128, conv_dim=64, c_dim=5, repeat_num=6):
        super(Discriminator, self).__init__()
        self.ngpu=torch.cuda.device_count()
        layers = []
        layers.append(nn.Conv2d(3, conv_dim, kernel_size=4, stride=2, padding=1))
        layers.append(nn.LeakyReLU(0.01))
         

        curr_dim = conv_dim
        for i in range(1, repeat_num):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1))
            layers.append(nn.LeakyReLU(0.01))
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num))
        self.main = nn.Sequential(*layers)
        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, bias=False)
        
    def forward(self, x):
        h = self.main(x)
        out_src = self.conv1(h)
        out_cls = self.conv2(h)
        # h1 clone removed..probably not needed
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))
"""

# logger

import tensorflow as tf


class Logger(object):
    """Tensorboard logger."""

    def __init__(self, log_dir):
        """Initialize summary writer."""
        # self.writer = tf.summary.FileWriter(log_dir) # depreciated
        # self.writer = tf.compat.v1.summary.FileWriter(log_dir) # new working one
        
        #self.writer = tf.contrib.summary.FileWriter(log_dir)
        #self.writer = tf.train.SummaryWriter(log_dir)
        
        # for version 2.xx
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Add scalar summary."""
        #summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        #self.writer.add_summary(summary, step)
        
        # tf version 2.x
        with self.writer.as_default():
            tf.summary.scalar(tag, value, step=step)
            self.writer.flush()

            
# brats_auc

import numpy as np
from glob import glob
from PIL import Image
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

imgs = glob("brats_syn_256_lambda0.1/results/*.jpg")
gts = [0]*975 + [1]*486

preds = []

for i in range(len(imgs)):
    im = np.array(Image.open("brats_syn_256_lambda0.1/results/{}-images.jpg".format(i+1)))
    rows = np.split(im, im.shape[0]//256, axis=0)

    for r in rows:
        cols = np.split(r, 5, axis=1)
        preds.append(np.max( np.mean(cols[1], axis=-1) ))

#print(roc_auc_score(gts, preds))

            
# Solver

#from model import Generator
#from model import Discriminator
from torch.autograd import Variable
from torchvision.utils import save_image
import torch
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from sklearn.cluster import KMeans
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torch.nn.parallel import DataParallel


class Solver(object):
    """Solver for training and testing Fixed-Point GAN."""

    def __init__(self, data_loader, config):
        """Initialize configurations."""

        # Data loader.
        self.data_loader = data_loader

        # Model configurations.
        self.c_dim = config.c_dim
        self.c2_dim = config.c2_dim
        self.image_size = config.image_size
        self.g_conv_dim = config.g_conv_dim
        self.d_conv_dim = config.d_conv_dim
        self.g_repeat_num = config.g_repeat_num
        self.d_repeat_num = config.d_repeat_num
        self.lambda_cls = config.lambda_cls
        self.lambda_rec = config.lambda_rec
        self.lambda_gp = config.lambda_gp
        self.lambda_id = config.lambda_id
        self.depth = config.depth

        # Training configurations.
        self.dataset = config.dataset
        self.batch_size = config.batch_size
        self.num_iters = config.num_iters
        self.num_iters_decay = config.num_iters_decay
        self.g_lr = config.g_lr
        self.d_lr = config.d_lr
        self.n_critic = config.n_critic
        self.beta1 = config.beta1
        self.beta2 = config.beta2
        self.resume_iters = config.resume_iters
        self.selected_attrs = config.selected_attrs

        # Test configurations.
        self.test_iters = config.test_iters

        # Miscellaneous.
        self.use_tensorboard = config.use_tensorboard
        #self.ngpu= config.ngpu
        self.ngpu= torch.cuda.device_count()
        self.device = torch.device('cuda' if (torch.cuda.is_available() and self.ngpu>0) else 'cpu')
        print('Number of Cuda devices: ',torch.cuda.device_count())

        # Directories.
        self.log_dir = config.log_dir
        self.sample_dir = config.sample_dir
        self.model_save_dir = config.model_save_dir
        self.result_dir = config.result_dir

        # Step size.
        self.log_step = config.log_step
        self.sample_step = config.sample_step
        self.model_save_step = config.model_save_step
        self.lr_update_step = config.lr_update_step

        self.rank=0
        self.world_size=1

        # Build the model and tensorboard.
        self.build_model()
        if self.use_tensorboard:
          self.build_tensorboard()
    
    def setup(self,rank,world_size):
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        # initialize the process group
        dist.init_process_group("nccl", rank=rank, world_size=world_size)

    def build_model(self):
        """Create a generator and a discriminator."""

        if self.dataset in ['CelebA', 'BRATS', 'Directory']:
            self.G = Generator(self.ngpu,self.g_conv_dim, self.c_dim, self.g_repeat_num,self.depth)
            self.D = Discriminator(self.ngpu,self.image_size, self.d_conv_dim, self.c_dim, self.d_repeat_num,self.depth) 

        #self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
        #self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])
        self.print_network(self.G, 'Generator')
        self.print_network(self.D, 'Discriminator')
    
        self.G.to(self.device)
        self.D.to(self.device)

        # Handle multi-gpu if desired
        if (self.device.type == 'cuda') and (self.ngpu > 1):
          # added for parallelism
          #for i in list(range(self.ngpu)):
          #  torch.cuda.set_device(i)       
          #  self.setup(self.rank,self.world_size)
          #  self.G = DistributedDataParallel(self.G, device_ids=[i],output_device=i)
          #  self.D = DistributedDataParallel(self.D, device_ids=[i],output_device=i)

          #self.setup(self.rank,self.world_size)
          #self.G = DistributedDataParallel(self.G, device_ids=list(range(self.ngpu)))
          #self.D = DistributedDataParallel(self.D, device_ids=list(range(self.ngpu)))
          
          self.G = DataParallel(self.G, device_ids=list(range(self.ngpu)))
          self.D = DataParallel(self.D, device_ids=list(range(self.ngpu)))

          self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
          self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])

        else:

          self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
          self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])

    def recreate_image(self, codebook, labels, w, h):
        """Recreate the (compressed) image from the code book & labels"""
        d = codebook.shape[1]
        image = np.zeros((w, h, d))
        label_idx = 0
        for i in range(w):
            for j in range(h):
                image[i][j] = codebook[labels[label_idx]]
                # orginal
                # label_idx += 1
                label_idx = label_idx + 1
        return image

    def print_network(self, model, name):
        """Print out the network information."""
        num_params = 0
        for p in model.parameters():
            # original
            # num_params += p.numel()
            num_params = num_params + p.numel()
        print(model)
        print(name)
        print("The number of parameters: {}".format(num_params))

    def restore_model(self, resume_iters):
        """Restore the trained generator and discriminator."""
        print('Loading the trained models from step {}...'.format(resume_iters))
        G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(resume_iters))
        D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(resume_iters))
        self.G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage)) # Load all tensors onto the CPU, using a function
        self.D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage)) # Load all tensors onto the CPU, using a function # to avoid GPU RAM surge when loading a model checkpoint.

    def build_tensorboard(self):
        """Build a tensorboard logger."""
        #from logger import Logger
        self.logger = Logger(self.log_dir)

    def update_lr(self, g_lr, d_lr):
        """Decay learning rates of the generator and discriminator."""
        for param_group in self.g_optimizer.param_groups:
            param_group['lr'] = g_lr
        for param_group in self.d_optimizer.param_groups:
            param_group['lr'] = d_lr

    def reset_grad(self):
        """Reset the gradient buffers."""
        self.g_optimizer.zero_grad()
        self.d_optimizer.zero_grad()

    def denorm(self, x):
        """Convert the range from [-1, 1] to [0, 1]."""
        out = (x + 1) / 2
        return out.clamp_(0, 1)

    def gradient_penalty(self, y, x):
        """Compute gradient penalty: (L2_norm(dy/dx) - 1)**2."""
        weight = torch.ones(y.size()).to(self.device)
        dydx = torch.autograd.grad(outputs=y,
                                   inputs=x,
                                   grad_outputs=weight,
                                   retain_graph=True,
                                   create_graph=True,
                                   only_inputs=True)[0]

        dydx = dydx.view(dydx.size(0), -1)
        dydx_l2norm = torch.sqrt(torch.sum(dydx**2, dim=1))
        return torch.mean((dydx_l2norm-1)**2)

    def label2onehot(self, labels, dim):
        """Convert label indices to one-hot vectors."""
        batch_size = labels.size(0)
        out = torch.zeros(batch_size, dim)
        out[np.arange(batch_size), labels.long()] = 1
        return out

    def create_labels(self, c_org, c_dim=5, dataset='CelebA', selected_attrs=None):
        """Generate target domain labels for debugging and testing."""
        # Get hair color indices.
        if dataset in ['CelebA']:
            hair_color_indices = []
            for i, attr_name in enumerate(selected_attrs):
                if attr_name in ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']:
                    hair_color_indices.append(i)

        c_trg_list = []
        for i in range(c_dim):
            if dataset in ['CelebA']:
                c_trg = c_org.clone()
                if i in hair_color_indices:  # Set one hair color to 1 and the rest to 0.
                    c_trg[:, i] = 1
                    for j in hair_color_indices:
                        if j != i:
                            c_trg[:, j] = 0
                else:
                    c_trg[:, i] = (c_trg[:, i] == 0)  # Reverse attribute value.
            elif dataset == 'BRATS':
                c_trg = c_org.clone()
                c_trg[:, i] = (c_trg[:, i] == 0)  # Reverse attribute value.
            elif dataset == 'Directory':
                c_trg = self.label2onehot(torch.ones(c_org.size(0))*i, c_dim)

            c_trg_list.append(c_trg.to(self.device))
        return c_trg_list

    def classification_loss(self, logit, target, dataset='CelebA'):
        """Compute binary or softmax cross entropy loss."""
        if dataset in ['CelebA', 'BRATS']:
            return F.binary_cross_entropy_with_logits(logit, target, reduction='sum') / logit.size(0)  # size_average=False inside binary_cross_entropy_with_logits
        elif dataset == 'Directory':
            return F.cross_entropy(logit, target)

    def create_grid(self, samples, img_files):
        """
        utility function to create a grid of GAN samples
        :param samples: generated samples for storing list[Tensors]
        :param img_files: list of names of files to write
        :return: None (saves multiple files)
        """
        from torchvision.utils import save_image
        from torch.nn.functional import interpolate
        from numpy import sqrt, power

        # dynamically adjust the colour of the images
        samples = [Generator.adjust_dynamic_range(sample) for sample in samples]

        # resize the samples to have same resolution:
        
        for i in range(len(samples)):
            samples[i] = interpolate(samples[i],
                                     scale_factor=power(2,self.depth - 1 - i))
        # save the images:
        for sample, img_file in zip(samples, img_files):
            save_image(sample, img_file, nrow=int(sqrt(sample.shape[0])),
                       normalize=True, scale_each=True, padding=0) 

    def train(self):
        """Train Fixed-Point GAN within a single dataset."""
        # Set data loader.
        if self.dataset in ['CelebA', 'BRATS', 'Directory']:
            data_loader = self.data_loader

        # Fetch fixed inputs for debugging.
        data_iter = iter(data_loader)
        x_fixed, c_org = next(data_iter)
        x_fixed = x_fixed.to(self.device)
        c_fixed_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)

        # Learning rate cache for decaying.
        g_lr = self.g_lr
        d_lr = self.d_lr

        # Start training from scratch or resume training.
        start_iters = 0
        if self.resume_iters:
            start_iters = self.resume_iters
            self.restore_model(self.resume_iters)

        # Start training.
        print('Start training...')
        start_time = time.time()
        for i in range(start_iters, self.num_iters):

            # =================================================================================== #
            #                             1. Preprocess input data                                #
            # =================================================================================== #
            

            # Fetch real images and labels.
            try:
                x_real, label_org = next(data_iter)
                #print("x_real",x_real.size()) # added to test the dimensions
                #print("label_org",label_org.size()) # added to test the dimensions
            except:
                data_iter = iter(data_loader)
                x_real, label_org = next(data_iter)

                #print("x_real: ",x_real.size()) # added to test the dimensions
                #print("label_org: ",label_org.size()) # added to test the dimensions

            # Generate target domain labels randomly.
            rand_idx = torch.randperm(label_org.size(0))
            label_trg = label_org[rand_idx]

            #print("rand_idx",rand_idx.size()) # added to test the dimensions
            #print("label_trg",label_trg.size()) # added to test the dimensions

            if self.dataset in ['CelebA', 'BRATS']:
                c_org = label_org.clone()
                c_trg = label_trg.clone()
            elif self.dataset == 'Directory':
                c_org = self.label2onehot(label_org, self.c_dim)
                c_trg = self.label2onehot(label_trg, self.c_dim)
            
            # added to output everything on 1 gpu 
            if (self.device.type == 'cuda') and (self.ngpu > 1):
              torch.cuda.set_device(0)   
            
            x_real = x_real.to(self.device)           # Input images.
            c_org = c_org.to(self.device)             # Original domain labels.
            c_trg = c_trg.to(self.device)             # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            #print("c_org",c_org.size()) # added to test the dimensions
            #print("c_targ",c_trg.size()) # added to test the dimensions
            

            # testing batch_size and image pyramid
            from torch.nn.functional import avg_pool2d
            batch_images=x_real  
            batch_images = [batch_images] + [avg_pool2d(batch_images, int(np.power(2, i)))
                                     for i in range(1, self.depth)]
                            
            #print('image length list: ',len(batch_images)) # added : equal number of depth or layers
            #def get_shapes(Depth):
            #  return [batch_images[i].size() for i in range(Depth)]
            #print('images shape:', get_shapes(self.depth) )


            # list needs to be added to a torch to be able to be used with to(self.device)
            #x_real=[_x_real.to(self.device) for _x_real in batch_images]    # input images
            #data = [_data.cuda() for _data in data] 
            #c_org = c_org.repeat(5,1,1).to(self.device)           # Original domain labels.
            #c_trg = c_trg.repeat(5,1,1).to(self.device)         # Target domain labels.
            #label_org = label_org.repeat(5,1,1).to(self.device)     # Labels for computing classification loss.
            #label_trg = label_trg.repeat(5,1,1).to(self.device)     # Labels for computing classification loss.

            #for _x_real in batch_images: 
            """
            def padding(ListofTensorImages):
              out=[]
              for img in ListofTensorImages:
                print(img.size())
                equ= (1024 - img.size(-1))/2
                print(equ)
                m = nn.ZeroPad2d(int(equ))
                out.append(m(img))
              return out

            ImagesWithPadding=padding(batch_images)
            for i in range(len(ImagesWithPadding)):
              print(ImagesWithPadding[i].size())

            stackedImages= torch.stack([ImagesWithPadding[0],ImagesWithPadding[1],ImagesWithPadding[2],ImagesWithPadding[3],ImagesWithPadding[4]],dim=0)
            """

            x_real_d=batch_images #.to(self.device)
            c_org = c_org.to(self.device)           # Original domain labels.
            c_trg = c_trg.to(self.device)         # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            #def get_shapes_01(Depth):
            #  return [batch_images[i].size() for i in range(Depth)],[c_org[i].size() for i in range(Depth)],[c_trg[i].size() for i in range(Depth)],[label_org[i].size() for i in range(Depth)],[label_trg[i].size() for i in range(Depth)]
            #print('batch_images, c_org, c_trig , label_org , label_trg :', get_shapes_01(depth__) )  

   
            # =================================================================================== #
            #                             2. Train the discriminator                              #
            # =================================================================================== #


          #  # Compute loss with real images.
            
            t0=time.time()
            out_src, out_cls = self.D(x_real_d)            
            d_loss_real = - torch.mean(out_src)
            d_loss_cls = self.classification_loss(out_cls, label_org, self.dataset)
            #print("Loss with real images",time.time()-t0)

            #print("out_src",out_src.size()) # added to test the dimensions
            #print("out_cls",out_cls.size()) # added to test the dimensions
            #print("d_loss_real",d_loss_real.size()) # added to test the dimensions
            #print("d_loss_cls",d_loss_cls.size()) # added to test the dimensions
          
          # Compute loss with fake images.
            t0=time.time()
            delta = self.G(x_real, c_trg) # c_trg is target class  # output 5 images : output the difference   
            
            #def get_shapes(Depth):
            #  return [delta[i].size() for i in range(Depth)]
            #print('images shape delta, results of Generator:', get_shapes(depth__) )
            #print("delta last batch",delta[-1].size())

            delta=delta[::-1] # inverse List
            x_fake=[torch.tanh(x_real_d[i] + delta[i]) for i in range(len(x_real_d))] # of 1 image 4x3x1024x1024
            x_fake=[x_fake[i].detach() for i in range(len(x_real_d))]
            out_src, out_cls = self.D(x_fake)
            d_loss_fake = torch.mean(out_src)
            #print("Loss with fake images",time.time()-t0)
            
            #print("out_src",out_src.size()) # added to test the dimensions
            #print("out_cls",out_cls.size()) # added to test the dimensions
            #print("d_loss_fake",d_loss_fake.size()) # added to test the dimensions
  

          # Compute loss for gradient penalty.
            t0=time.time()
            alpha = torch.rand(x_real.size(0), 1, 1, 1).to(self.device) # x_real.size(0) is the nb of batch
            #x_hat = (alpha * x_real.data + (1 - alpha) * x_fake.data).requires_grad_(True) 
            x_hat = [(alpha * x_real_d[i].data + (1 - alpha) * x_fake[i].data).requires_grad_(True) for i in range(len(x_real_d))]
            out_src, _ = self.D(x_hat)
            d_loss_gp = self.gradient_penalty(out_src, x_hat)
            #print("Loss for gredient penalty",time.time()-t0)
            
            #check the gpu before changing
            #print(d_loss_gp.get_device())

            #print("alpha, compute gradient penalty",alpha) # added to test the dimensions
            #print("out_src",out_src.size()) # added to test the dimensions
            #print("d_loss_gp",d_loss_gp) # added to test the dimensions
    

            # Backward and optimize.#
            d_loss = d_loss_real + d_loss_fake + self.lambda_cls * d_loss_cls + self.lambda_gp * d_loss_gp 

            #print("d_loss",d_loss) # added to test the dimensions
            
            t0=time.time()
            self.reset_grad()

            if (self.device.type == 'cuda') and (self.ngpu > 1):
              d_loss.mean().backward()
            else:
              d_loss.backward()

            self.d_optimizer.step()
            #print("back propagation and optimization",time.time()-t0)
            # Logging.
            loss = {}
            loss['D/loss_real'] = d_loss_real.item()
            loss['D/loss_fake'] = d_loss_fake.item()
            loss['D/loss_cls'] = d_loss_cls.item()
            loss['D/loss_gp'] = d_loss_gp.item()
            # added
            loss['d_loss'] = d_loss.item()
            
            # =================================================================================== #
            #                               3. Train the generator                                #
            # =================================================================================== #
            
            if (i+1) % self.n_critic == 0:

                # Original-to-target domain.
                t0=time.time()
                delta = self.G(x_real, c_trg)
                delta=delta[::-1]
                x_fake=[torch.tanh(x_real_d[i] + delta[i]) for i in range(len(x_real_d))] # of 1 image 4x3x1024x1024
                #x_fake=[x_fake[i].detach() for i in range(len(x_real_d))]
                out_src, out_cls = self.D(x_fake)
                #x_fake = torch.tanh(x_real_d + delta)
                #out_src, out_cls = self.D(x_fake)
                g_loss_fake = - torch.mean(out_src)
                g_loss_cls = self.classification_loss(out_cls, label_trg, self.dataset)
                
                #print("Original-to-target domain : Generator",time.time()-t0)

                #print("delta, generator",delta.size()) # added to test the dimensions
                #print("x_fake: tanh(x_real + delta)",x_fake.size()) # added to test the dimensions
                #print("out_src",out_src.size()) # added to test the dimensions
                #print("out_cls",out_cls.size()) # added to test the dimensions
                #print("g_loss_fake",g_loss_fake) # added to test the dimensions
                #print("g_loss_cls",g_loss_cls) # added to test the dimensions



                # Original-to-original domain.
                t0=time.time()

                delta_id = self.G(x_real, c_org)
                delta_id=delta_id[::-1]
                #x_fake_id = torch.tanh(x_real_d + delta_id)
                x_fake_id=[torch.tanh(x_real_d[i] + delta_id[i]) for i in range(len(x_real_d))]
                out_src_id, out_cls_id = self.D(x_fake_id)
                g_loss_fake_id = - torch.mean(out_src_id)
                g_loss_cls_id = self.classification_loss(out_cls_id, label_org, self.dataset)
                #g_loss_id = torch.mean(torch.abs(x_real - torch.tanh(delta_id + x_real)))

                
                def image_padding(ListofTensorImages):
                  out=[]
                  for img in ListofTensorImages:
                    #print(img.size())
                    equ= (1024 - img.size(-1))/2
                    #print(equ)
                    m = nn.ZeroPad2d(int(equ))
                    out.append(m(img))
                  return out

                #g_loss_id = torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_fake_id[i]) for i in range(len(x_real_d))]))) # 5 scaled images
                g_loss_id = torch.mean(torch.abs(x_real_d[0] - x_fake_id[0])) # 1 1024x1024 image
                #print("Original-to-original domain : Generator",time.time()-t0)

                #print("delta_id, generator",delta_id.size()) # added to test the dimensions
                #print("x_fake_id: tanh(x_real + delta_id)",x_fake_id.size()) # added to test the dimensions
                #print("out_src_id",out_src_id.size()) # added to test the dimensions
                #print("out_cls_id",out_cls_id.size()) # added to test the dimensions
                #print("g_loss_fake_id",g_loss_fake_id) # added to test the dimensions
                #print("g_loss_cls_id",g_loss_cls_id) # added to test the dimensions
                #print("g_loss_id",g_loss_id) # added to test the dimensions


                # Target-to-original domain.
                t0=time.time()

                #print("x_fake[-1] size: ",x_fake[0].size()) # added to test the dimensions
                delta_reconst = self.G(x_fake[0], c_org)
                delta_reconst=delta_reconst[::-1]
                #print("delta_reconst[-1] size: ",delta_reconst[-1].size()) # added to test the dimensions


                x_reconst=[torch.tanh(x_fake[i] + delta_reconst[i]) for i in range(len(x_real_d))]
                #x_reconst = torch.tanh(x_fake + delta_reconst)
                #g_loss_rec = torch.mean(torch.abs(x_real - x_reconst))
                
                #g_loss_rec= torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_reconst[i]) for i in range(len(x_real_d))]))) # 5 scaled images
                g_loss_rec= torch.mean(torch.abs(x_real_d[0] - x_reconst[0])) # 1 1024x1024 image


                #print("delta_reconst, generator",delta_reconst.size()) # added to test the dimensions
                #print("x_reconst: tanh(x_real + delta_reconst)",x_reconst.size()) # added to test the dimensions
                #print("g_loss_rec",g_loss_rec) # added to test the dimensions

                #print("Target-to-original domain : Generator",time.time()-t0)

                # Original-to-original domain.
                t0=time.time()

                delta_reconst_id = self.G(x_fake_id[0], c_org) 
                delta_reconst_id = delta_reconst_id[::-1]
                x_reconst_id=[torch.tanh(x_fake_id[i] + delta_reconst_id[i]) for i in range(len(x_real_d))]
                #x_reconst_id = torch.tanh(x_fake_id + delta_reconst_id)
                #g_loss_rec_id = torch.mean(torch.abs(x_real - x_reconst_id))

                #g_loss_rec_id= torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_reconst_id[i]) for i in range(len(x_real_d))])))  # 5 scales images 
                g_loss_rec_id= torch.mean(torch.abs(x_real_d[0] - x_reconst_id[0]))   # 1 1024x1024 images 




                #print("delta_reconst_id, generator",delta_reconst_id.size()) # added to test the dimensions
                #print("x_reconst_id: tanh(x_real + delta_reconst_id)",x_reconst_id.size()) # added to test the dimensions
                #print("g_loss_rec_id",g_loss_rec_id) # added to test the dimensions

                #print("Original-to-original domain : Generator",time.time()-t0)


                # Backward and optimize.
                g_loss_same = g_loss_fake_id + self.lambda_rec * g_loss_rec_id + self.lambda_cls * g_loss_cls_id + self.lambda_id * g_loss_id
                g_loss = g_loss_fake + self.lambda_rec * g_loss_rec + self.lambda_cls * g_loss_cls + g_loss_same

                #print("g_loss_same",g_loss_same.size()) # added to test the dimensions
                #print("g_loss",g_loss) # added to test the dimensions

                t0=time.time()
                self.reset_grad()
                #g_loss.backward()

                if (self.device.type == 'cuda') and (self.ngpu > 1):
                  g_loss.mean().backward()
                else:
                  g_loss.backward()

                self.g_optimizer.step()
                #print("back Propagation: Generator",time.time()-t0)

                # Logging.
                loss['G/loss_fake'] = g_loss_fake.item()
                loss['G/loss_rec'] = g_loss_rec.item()
                loss['G/loss_cls'] = g_loss_cls.item()
                loss['G/loss_fake_id'] = g_loss_fake_id.item()
                loss['G/loss_rec_id'] = g_loss_rec_id.item()
                loss['G/loss_cls_id'] = g_loss_cls_id.item()
                loss['G/loss_id'] = g_loss_id.item()
                # added
                loss['g_loss'] = g_loss.item()

            # =================================================================================== #
            #                                 4. Miscellaneous                                    #
            # =================================================================================== #

            # Print out training information.
            if (i+1) % self.log_step == 0:
                et = time.time() - start_time
                et = str(datetime.timedelta(seconds=et))[:-7]
                log = "Elapsed [{}], Iteration [{}/{}]".format(et, i+1, self.num_iters)
                for tag, value in loss.items():
                    log += ", {}: {:.4f}".format(tag, value)
                print(log,flush=True)

                if self.use_tensorboard:
                    for tag, value in loss.items():
                        self.logger.scalar_summary(tag, value, i+1)

            # Translate fixed images for debugging.
            if (i+1) % self.sample_step == 0:
              from torch.nn.functional import avg_pool2d
              #batch_images_fixed=x_fixed  
              #depth__=5
              #batch_images_fixed = [batch_images_fixed] + [avg_pool2d(batch_images_fixed, int(np.power(2, i)))
              #                       for i in range(1, depth__)]
              with torch.no_grad():
                  x_fake_list = [x_fixed]
                  print("fake_list[0]",x_fake_list[0].size()) # 3x3x1024x1024   
                  for c_fixed in c_fixed_list:
                      delta = self.G(x_fixed, c_fixed)
                      delta = delta[::-1]
                      gen_img=torch.tanh(delta[0] + x_fixed)
                      x_fake_list.append(gen_img) # add to list
                  x_concat = torch.cat(x_fake_list, dim=3)
                  sample_path = os.path.join(self.sample_dir, '{}-images.jpg'.format(i+1))
                  save_image(self.denorm(x_concat.data.cpu()), sample_path, nrow=1, padding=0)
                  print('Saved real and fake images into {}...'.format(sample_path))


                  # create a grid of samples and save it
                  #reses = [str(int(np.power(2, dep+4))) + "_x_"
                  #+ str(int(np.power(2, dep+4)))
                  #for dep in reversed(range(2,depth__ + 2))]

                  #gen_img_files = [os.path.join(self.sample_dir, res, "gen_" +
                  #                              str(i) + ".png")
                  #                  for res in reses]

                  # Make sure all the required directories exist
                  # otherwise make them
                  #os.makedirs(self.sample_dir, exist_ok=True)
                  #for gen_img_file in gen_img_files:
                  #    os.makedirs(os.path.dirname(gen_img_file), exist_ok=True)

                  # fake list should contain the x_fake_list of different versions
                  #print("x_fixed",x_fixed[0].size()) 
                  #print("delta",delta[0].size())      
                  #gen_imgs=[torch.tanh(batch_images_fixed[i] + delta[i]) for i in range(len(x_real_d))]
                  #self.create_grid(gen_imgs,gen_img_files)

            # Save model checkpoints.
            if (i+1) % self.model_save_step == 0:
                G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(i+1))
                D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(i+1))
                
                if (self.device.type == 'cuda') and (self.ngpu > 1):
                  torch.save(self.G.module.state_dict(), G_path) # loading doesn't change
                  torch.save(self.D.module.state_dict(), D_path) # loading doesn't change
                else:
                  torch.save(self.G.state_dict(), G_path)
                  torch.save(self.D.state_dict(), D_path)

                print('Saved model checkpoints into {}...'.format(self.model_save_dir))

            # Decay learning rates.
            if (i+1) % self.lr_update_step == 0 and (i+1) > (self.num_iters - self.num_iters_decay):
                g_lr = g_lr - (self.g_lr / float(self.num_iters_decay))
                d_lr = d_lr - (self.d_lr / float(self.num_iters_decay))
                #original
                #g_lr -= (self.g_lr / float(self.num_iters_decay))
                #d_lr -= (self.d_lr / float(self.num_iters_decay))
                self.update_lr(g_lr, d_lr)
                print ('Decayed learning rates, g_lr: {}, d_lr: {}.'.format(g_lr, d_lr))


    def test(self):
        """Translate images using Fixed-Point GAN trained on a single dataset."""
        # Load the trained generator.
        self.restore_model(self.test_iters)
        
        # Set data loader.
        if self.dataset in ['CelebA', 'Directory']:
            data_loader = self.data_loader
        
        with torch.no_grad():
            for i, (x_real, c_org) in enumerate(data_loader):

                # Prepare input images and target domain labels.
                x_real = x_real.to(self.device)
                c_trg_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)

                # Translate images.
                x_fake_list = [x_real]
                for c_trg in c_trg_list:
                    x_fake_list.append(torch.tanh(x_real + self.G(x_real, c_trg)))

                # Save the translated images.
                x_concat = torch.cat(x_fake_list, dim=3)
                result_path = os.path.join(self.result_dir, '{}-images.jpg'.format(i+1))
                save_image(self.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
                print('Saved real and fake images into {}...'.format(result_path))



    def test_brats(self):
        """Translate images using Fixed-Point GAN trained on a single dataset."""
        # Load the trained generator.
        self.restore_model(self.test_iters)
        
        # Set data loader.
        if self.dataset in ['BRATS']:
            data_loader = self.data_loader
        
        with torch.no_grad():
            for i, (x_real, c_org) in enumerate(data_loader):
                x_real = x_real.to(self.device)

                c_trg = c_org.clone()
                c_trg[:, 0] = 0 # always to healthy              
                c_trg_list = [c_trg.to(self.device)]

                # Translate images.
                x_fake_list = [x_real]
                for c_trg in c_trg_list:
                    delta = self.G(x_real, c_trg)
                    delta_org = torch.abs(torch.tanh(delta + x_real) - x_real) - 1.0
                    delta_gray = np.mean(delta_org.data.cpu().numpy(), axis=1)
                    delta_gray_norm = []

                    loc = []
                    cls_mul = []

                    for indx in range(delta_gray.shape[0]):
                        temp = delta_gray[indx, :, :] + 1.0  
                        tempimg_th = np.percentile(temp, 99)
                        tempimg = np.float32(temp >= tempimg_th)
                        temploc = np.reshape(tempimg, (self.image_size*self.image_size, 1))

                        kmeans = KMeans(n_clusters=2, random_state=0).fit(temploc)
                        labels = kmeans.predict(temploc)

                        recreated_loc = self.recreate_image(kmeans.cluster_centers_, labels, self.image_size, self.image_size)
                        recreated_loc = ((recreated_loc - np.min(recreated_loc)) / (np.max(recreated_loc) - np.min(recreated_loc)))

                        loc.append(recreated_loc)
                        delta_gray_norm.append( tempimg )


                    loc = np.array(loc, dtype=np.float32)[:, :, :, 0]
                    delta_gray_norm = np.array(delta_gray_norm)

                    loc = (loc * 2.0) - 1.0
                    delta_gray_norm = (delta_gray_norm * 2.0) - 1.0

                    x_fake_list.append( torch.from_numpy(np.repeat(delta_gray[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # difference map
                    x_fake_list.append( torch.from_numpy(np.repeat(delta_gray_norm[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # localization thershold
                    x_fake_list.append( torch.from_numpy(np.repeat(loc[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # localization kmeans
                    x_fake_list.append( torch.tanh(delta + x_real) ) # generated image

                # Save the translated images.
                x_concat = torch.cat(x_fake_list, dim=3)
                result_path = os.path.join(self.result_dir, '{}-images.jpg'.format(i+1))
                save_image(self.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
                print('Saved real and fake images into {}...'.format(result_path))



    def cleanup():
      dist.destroy_process_group()   

# Solver               

from torch.utils import data
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from PIL import Image
import torch
import os
import random
from glob import glob


class CelebA(data.Dataset):
    """Dataset class for the CelebA dataset."""

    def __init__(self, image_dir, attr_path, selected_attrs, transform, mode):
        """Initialize and preprocess the CelebA dataset."""
        self.image_dir = image_dir
        self.attr_path = attr_path
        self.selected_attrs = selected_attrs
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.test_dataset = []
        self.attr2idx = {}
        self.idx2attr = {}
        self.preprocess()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        else:
            self.num_images = len(self.test_dataset)

    def preprocess(self):
        """Preprocess the CelebA attribute file."""
        lines = [line.rstrip() for line in open(self.attr_path, 'r')]
        all_attr_names = lines[1].split()
        for i, attr_name in enumerate(all_attr_names):
            self.attr2idx[attr_name] = i
            self.idx2attr[i] = attr_name

        lines = lines[2:]
        random.seed(1234)
        random.shuffle(lines)
        for i, line in enumerate(lines):
            split = line.split()
            filename = split[0]
            values = split[1:]

            label = []
            for attr_name in self.selected_attrs:
                idx = self.attr2idx[attr_name]
                label.append(values[idx] == '1')

            if (i+1) < 2000:
                self.test_dataset.append([filename, label])
            else:
                self.train_dataset.append([filename, label])

        print('Finished preprocessing the CelebA dataset...')

    def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.test_dataset
        filename, label = dataset[index]

        # when gcp is used 
        os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
        Bucket_name=config.GCP_Bucket_name
        # Use Key authentication
        storage_client = storage.Client()
        # create bucket
        bucket = storage_client.get_bucket(Bucket_name)

        # 'data/celeba/images/'
        # data_hd/CelebA/
        blob = bucket.blob('data_hd/CelebA/'+filename)
        filename__=blob.download_as_string()
        image=Image.open(BytesIO(filename__))

        # when gcp is not used
        #image = Image.open(os.path.join(self.image_dir, filename))
        return self.transform(image), torch.FloatTensor(label)

    def __len__(self):
        """Return the number of images."""
        return self.num_images


class BRATS_SYN(data.Dataset):
    """Dataset class for the BRATS dataset."""

    def __init__(self, image_dir, transform, mode):
        """Initialize and Load the BRATS dataset."""
        self.image_dir = image_dir
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.test_dataset = []
        self.load_data()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        else:
            self.num_images = len(self.test_dataset)

    def load_data(self):
        """Load BRATS dataset"""
        
        # Load test dataset
        test_neg = glob(os.path.join(self.image_dir, 'test', 'negative', '*jpg'))
        test_pos = glob(os.path.join(self.image_dir, 'test', 'positive', '*jpg'))

        for filename in test_neg:
            self.test_dataset.append([filename, [0]])

        for filename in test_pos:
            self.test_dataset.append([filename, [1]])


        # Load train dataset
        train_neg = glob(os.path.join(self.image_dir, 'train', 'negative', '*jpg'))
        train_pos = glob(os.path.join(self.image_dir, 'train', 'positive', '*jpg'))

        for filename in train_neg:
            self.train_dataset.append([filename, [0]])

        for filename in train_pos:
            self.train_dataset.append([filename, [1]])

        print('Finished loading the BRATS dataset...')

    def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.test_dataset
        filename, label = dataset[index]
        image = Image.open(filename)
        return self.transform(image), torch.FloatTensor(label)

    def __len__(self):
        """Return the number of images."""
        return self.num_images


def get_loader(image_dir, attr_path, selected_attrs, crop_size=178, image_size=128, 
               batch_size=16, dataset='CelebA', mode='train', num_workers=1):
    """Build and return a data loader."""
    transform = []
    if mode == 'train':
        transform.append(T.RandomHorizontalFlip())
    transform.append(T.CenterCrop(crop_size))
    transform.append(T.Resize(image_size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    if dataset == 'CelebA':
        dataset =  CelebA(image_dir, attr_path, selected_attrs, transform, mode)
    elif dataset == 'BRATS':
        dataset = BRATS_SYN(image_dir, transform, mode)
    elif dataset == 'Directory':
        dataset = ImageFolder(image_dir, transform)

    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
    return data_loader

                                
                
# Main

import os
import argparse
#from solver import Solver
#from data_loader import get_loader
from torch.backends import cudnn
from google.cloud import storage
import tensorflow as tf
import time
from io import BytesIO
#from PIL import Image
import zipfile
from glob import glob
from tensorboard import version

# detect directory
# current directory: path at terminal when executing this file
cwd = os.getcwd()
print("Curret Directory: ",cwd)


def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training.
    cudnn.benchmark = True #True
    cudnn.enabled = True # added
    torch.cuda.empty_cache()
    

    # Create directories if not exist.
    if not os.path.exists(config.log_dir):
        os.makedirs(config.log_dir)
    print(os.path.exists(config.log_dir))
    if not os.path.exists(config.model_save_dir):
        os.makedirs(config.model_save_dir)
    print(os.path.exists(config.model_save_dir))
    if not os.path.exists(config.sample_dir):
        os.makedirs(config.sample_dir)
    print(os.path.exists(config.sample_dir))
    if not os.path.exists(config.result_dir):
        os.makedirs(config.result_dir)
    print(os.path.exists(config.result_dir))

    #os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
    Bucket_name=config.GCP_Bucket_name

    # Use Key authentication
    #storage_client = storage.Client()

    # create bucket
    #bucket = storage_client.get_bucket(Bucket_name)

    # read dataset.zip from bucket and copy it to directory
    #t0=time.time()
    #for blob in bucket.list_blobs(prefix= config.dataset_in_cloud):
    #  print(blob.name)
    #  tempfile= cwd + "/dataset__gcp.zip"
    #  blob.download_to_filename(tempfile)
    #t_end_zip=time.time()-t0
    #print('dataset gcp moved to local')

    #time.sleep(1)

    # read from directort zip file and extract
    #t0=time.time()
    #with zipfile.ZipFile("dataset__gcp.zip","r") as zip_ref:
    #    zip_ref.extractall(config.dataset_folder_name)
    #t_end_unzip=time.time()-t0
    #print('Extraction completed')

    #time.sleep(1)


        
    # Data loader.
    data_loader = None

    # 'data/celeba/images'
    # config.image_dir

    if config.dataset in ['CelebA']:#CelebA
        data_loader = get_loader(config.image_dir, config.attr_path, config.selected_attrs,
                                   config.crop_size, config.image_size, config.batch_size,
                                   'CelebA', config.mode, config.num_workers)


    elif config.dataset in ['BRATS']:
        data_loader = get_loader('{}/brats/syn'.format(config.dataset_folder_name), None, None,
                                   config.crop_size, config.image_size, config.batch_size,
                                   'BRATS', config.mode, config.num_workers)


    elif config.dataset in ['Directory']:
        data_loader = get_loader('config.image_dir', None, None,
                                 config.crop_size, config.image_size, config.batch_size,
                                 'Directory', config.mode, config.num_workers)

        
    # Solver for training and testing Fixed-Point GAN.
    solver = Solver(data_loader, config)

    if config.mode == 'train':
        if config.dataset in ['CelebA', 'BRATS', 'Directory']:  #CelebA
            solver.train()
    elif config.mode == 'test':
        if config.dataset in ['CelebA', 'Directory']:
            solver.test()
    elif config.mode == 'test_brats':
        if config.dataset in ['BRATS']:
            solver.test_brats()

    def Copy_local_dir_to_GCP_bucket(directory,folder_name_to_be_copied,gcp_output_folder):
        
        # to be later on fixed to be controlled from the main
        os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
        Bucket_name=config.GCP_Bucket_name
        # Use Key authentication
        storage_client = storage.Client()
        # create bucket
        bucket = storage_client.get_bucket(Bucket_name)

        # Get the list of all files in directory tree at given path
        path = directory + "/" + folder_name_to_be_copied + "/"
        #we shall store all the file names in this list
        filelist = []

        for root, dirs, files in os.walk(path):
          for file in files:
            #append the file name to the list
            filelist.append(os.path.join(root,file))

        # write files and subdirectories to gcp
        for name in filelist:
            # print(name)
            blob=bucket.blob(name.replace(directory+"/" , gcp_output_folder+"/"))
            blob.upload_from_filename(name)
        return print('Output were transfered to gcp bucket')


    if config.dataset == 'BRATS':
      t0=time.time()
      Copy_local_dir_to_GCP_bucket(cwd,'brats_syn_256_lambda0.1', bucket,"Output")
      t_end_copy_folder_to_gcp=time.time()-t0
      print('time to end copying folders to gcp bucket',t_end_copy_folder_to_gcp)
    elif config.dataset == 'CelebA': # CelebA
      t0=time.time()
      Copy_local_dir_to_GCP_bucket(cwd,'celeba_1',"Output")
      t_end_copy_folder_to_gcp=time.time()-t0
      print('time to end copying folders to gcp bucket',t_end_copy_folder_to_gcp)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Model configuration.
    parser.add_argument('--c_dim', type=int, default=5, help='dimension of domain labels (1st dataset)')
    parser.add_argument('--c2_dim', type=int, default=8, help='dimension of domain labels (2nd dataset)')
    parser.add_argument('--crop_size', type=int, default=1024, help='crop size for the images') # was 178
    parser.add_argument('--image_size', type=int, default=128, help='image resolution')
    parser.add_argument('--g_conv_dim', type=int, default=16, help='number of conv filters in the first layer of G') # was 64
    parser.add_argument('--d_conv_dim', type=int, default=16, help='number of conv filters in the first layer of D') # was 64
    parser.add_argument('--g_repeat_num', type=int, default=6, help='number of residual blocks in G') # ws 6
    parser.add_argument('--d_repeat_num', type=int, default=8, help='number of strided conv layers in D') # was 6
    parser.add_argument('--lambda_cls', type=float, default=1, help='weight for domain classification loss')
    parser.add_argument('--lambda_rec', type=float, default=10, help='weight for reconstruction loss')
    parser.add_argument('--lambda_gp', type=float, default=10, help='weight for gradient penalty')
    parser.add_argument('--lambda_id', type=float, default=10, help='weight for identity loss')
    parser.add_argument('--depth', type=int, default=5, help='Number of output images/layers')

    # Training configuration.
    parser.add_argument('--dataset', type=str, default='CelebA', choices=['CelebA', 'BRATS', 'Directory'])#CelebA
    parser.add_argument('--batch_size', type=int, default=16, help='mini-batch size')
    parser.add_argument('--num_iters', type=int, default=200000, help='number of total iterations for training D')
    parser.add_argument('--num_iters_decay', type=int, default=100000, help='number of iterations for decaying lr')
    parser.add_argument('--g_lr', type=float, default=0.0001, help='learning rate for G')
    parser.add_argument('--d_lr', type=float, default=0.0001, help='learning rate for D')
    parser.add_argument('--n_critic', type=int, default=5, help='number of D updates per each G update')
    parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
    parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
    parser.add_argument('--resume_iters', type=int, default=None, help='resume training from this step')
    parser.add_argument('--selected_attrs', '--list', nargs='+', help='selected attributes for the CelebA dataset',
                        default=['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young'])

    # Test configuration.
    parser.add_argument('--test_iters', type=int, default=200000, help='test model from this step')

    # Miscellaneous.
    parser.add_argument('--num_workers', type=int, default=1)
    parser.add_argument('--mode', type=str, default='train', choices=['train', 'test', 'test_brats'])
    parser.add_argument('--use_tensorboard', type=str2bool, default=True)
    parser.add_argument('--job-dir', help=" will be passed through from the AI-Platform to your program when you define it.")
    #parser.add_argument('--ngpu', type=int, default=0)
    parser.add_argument('--GCP_project_id', type=str, default='fpcityuni-2020')
    parser.add_argument('--GCP_Bucket_name', type=str, default="fpcityuni-2020-storage")
    
    # Directories.
    parser.add_argument('--image_dir', type=str, default='data/celeba/images')
    
    parser.add_argument('--dataset_in_cloud', type=str, default='dataset.zip')
    parser.add_argument('--dataset_folder_name', type=str, default='data')
    parser.add_argument('--attr_path', type=str, default='data/celeba/list_attr_celeba.txt')
    parser.add_argument('--log_dir', type=str, default='celeba/logs')
    parser.add_argument('--model_save_dir', type=str, default='celeba/models')
    parser.add_argument('--sample_dir', type=str, default='celeba/samples')
    parser.add_argument('--result_dir', type=str, default='celeba/results')

    # Step size.
    parser.add_argument('--log_step', type=int, default=10)
    parser.add_argument('--sample_step', type=int, default=1000)
    parser.add_argument('--model_save_step', type=int, default=10000)
    parser.add_argument('--lr_update_step', type=int, default=1000)

    config = parser.parse_args()
    print(config, flush='True')
    print('__Tensorflow VERSION',tf.__version__)
    print('__Tensorboard VERSION:', version.VERSION)
    print('__pyTorch VERSION:', torch.__version__)
    #import torchvision
    #print('__Torchvision VERSION',torchvision.__version__)
    print('__CUDA VERSION',torch.version.cuda )
    from subprocess import call
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__Devices')
    if torch.cuda.device_count() >0:
      print('Active CUDA Device: GPU', torch.cuda.current_device())
      print('Current device name',torch.cuda.get_device_name(0))
    else:
      print('No Cuda Device')
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    torch.autograd.set_detect_anomaly(True)
    main(config)
    print(torch.cuda.memory_allocated())


Overwriting GAN_nGPU_GCP_Method1_LastResolutionLoss_v2.py


# Method 1 Execution

In [6]:
! python3 -u GAN_nGPU_GCP_Method1_LastResolutionLoss_v2.py --mode train --dataset 'CelebA' --image_size 1024 --crop_size 1024 --c_dim 5 --g_conv_dim 16 --d_conv_dim 16 --depth 4 --d_repeat_num 8 --g_repeat_num 6 \
                 --GCP_project_id fpcityuni-2020 \
                 --GCP_Bucket_name fpcityuni-2020-storage \
                 --image_dir data/test_max_size_of_readable_images \
                 --attr_path list_attr_celeba_hd.txt \
                 --sample_dir celeba_1/samples \
                 --log_dir celeba_1/logs \
                 --model_save_dir celeba_1/models \
                 --result_dir celeba_1/results \
                 --selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young --lambda_id 10 \
                 --batch_size 3 --num_workers 3 --num_iters 50000 --log_step 50 --sample_step 5000 #--resume_iters 10000

2020-08-15 21:29:31.710526: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Curret Directory:  /content/drive/My Drive/FinalProject-GCP
Namespace(GCP_Bucket_name='fpcityuni-2020-storage', GCP_project_id='fpcityuni-2020', attr_path='list_attr_celeba_hd.txt', batch_size=3, beta1=0.5, beta2=0.999, c2_dim=8, c_dim=5, crop_size=1024, d_conv_dim=16, d_lr=0.0001, d_repeat_num=8, dataset='CelebA', dataset_folder_name='data', dataset_in_cloud='dataset.zip', depth=4, g_conv_dim=16, g_lr=0.0001, g_repeat_num=6, image_dir='data/test_max_size_of_readable_images', image_size=1024, job_dir=None, lambda_cls=1, lambda_gp=10, lambda_id=10.0, lambda_rec=10, log_dir='celeba_1/logs', log_step=50, lr_update_step=1000, mode='train', model_save_dir='celeba_1/models', model_save_step=10000, n_critic=5, num_iters=50000, num_iters_decay=100000, num_workers=3, result_dir='celeba_1/results', resume_iters=None, sample_dir='celeba_1/samples', samp

#Method 2:

In [8]:
%%writefile GAN_nGPU_GCP_Method2_scaledimagesaveraging_v2.py

# inserting a file with huge number of images doesn't work with google drive, gcp is needed to read the data from there
# Model
# works with 2 gpu not 3 and with 20 batch size 10.30 mins

# Change the code to blocks

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.nn import ModuleList, Conv2d


class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout, kernels_per_layer=1, K_depth=3, S_depth=1, P_depth=1, D_depth=1, K_point=1, S_point=1, P_point=0, D_point=1, bias=False):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=K_depth,stride=S_depth, padding=P_depth,dilation=D_depth, groups=nin,bias=bias)
        self.pointwise = nn.Conv2d(nin * kernels_per_layer, nout, kernel_size=K_point,stride=S_point,padding=P_point,dilation=D_point, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class ResidualBlock(nn.Module):
    """Residual Block with instance normalization."""
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            depthwise_separable_conv(nin=dim_in,nout=dim_out, kernels_per_layer=1,K_depth=3,S_depth=1,P_depth=1,D_depth=1,K_point=1,S_point=1,P_point=0,D_point=1,bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            depthwise_separable_conv(nin=dim_in,nout=dim_out, kernels_per_layer=1,K_depth=3,S_depth=1,P_depth=1,D_depth=1,K_point=1,S_point=1,P_point=0,D_point=1,bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)


class Generator(nn.Module):
    """Generator network."""
    def __init__(self, ngpu=0, conv_dim=16, c_dim=1, repeat_num=6,depth=4):
        super(Generator, self).__init__()
        self.ngpu = torch.cuda.device_count()
        
        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True))

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(depth):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))
        
        self.main_DownSampling_bottleneck = nn.Sequential(*layers)
        
        # create a module list of the other required general convolution blocks
        self.UpsamplingLayers = ModuleList()
        self.rgb_converters = ModuleList()

        # Up-sampling layers.
        def conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential( nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=K, stride=S, padding=P, bias=bias_),
            nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True))
        
        # to rgb conversion
        def to_rgb(in_channels):
          layers_rgb = []
          layers_rgb.append(nn.Sequential(Conv2d(in_channels, 3, (1, 1),(1,1),(0,0), bias=True)))
          #layers_rgb.append(nn.Tanh())
          out = nn.Sequential(*layers_rgb)
          return out

        for i in range(depth):
          if i != (depth-1):
            self.UpsamplingLayers.append(conv_block(curr_dim,4,2,1,False))
            rgb = to_rgb(int(curr_dim//2))
            self.rgb_converters.append(rgb)
            curr_dim = curr_dim // 2
          else :
            self.UpsamplingLayers.append(conv_block(curr_dim,4,2,1,False)) # curr_dim,7,1,3,False)
            rgb = to_rgb(int(curr_dim//2))
            self.rgb_converters.append(rgb)
            curr_dim = curr_dim // 2
          

    def forward(self, x, c):
        # Replicate spatially and concatenate domain information.
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)

        #print(self.UpsamplingLayers)
        #print(self.rgb_converters)

        outputs=[]
        Y=self.main_DownSampling_bottleneck(x) # start the computational pipeline
        #counter=0
        #print("Y after downsmapling and bottleneck",Y.size())
        for block, converter in zip(self.UpsamplingLayers, self.rgb_converters):
            #print("Y again",Y.size())
            Y = block(Y)
            outputs.append(converter(Y))
            #counter=counter+1
            #print(counter)
        return outputs

    #@staticmethod
    def adjust_dynamic_range(data, drange_in=(-1, 1), drange_out=(0, 1)):
      """
      adjust the dynamic colour range of the given input data
      :param data: input image data
      :param drange_in: original range of input
      :param drange_out: required range of output
      :return: img => colour range adjusted images
      """
      if drange_in != drange_out:
        scale = (np.float32(drange_out[1]) - np.float32(drange_out[0])) / (np.float32(drange_in[1]) - np.float32(drange_in[0]))
        bias = (np.float32(drange_out[0]) - np.float32(drange_in[0]) * scale)
        data = data * scale + bias
      return torch.clamp(data, min=0, max=1)




# Discriminator code using blocks

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.nn import ModuleList, Conv2d


class Discriminator(nn.Module):
    """Discriminator network with PatchGAN."""
    def __init__(self, ngpu=0, image_size=1024, conv_dim=16, c_dim=5, repeat_num=8,depth=4):
        super(Discriminator, self).__init__()
        self.ngpu=torch.cuda.device_count()

        def from_rgb(out_channels):
          return Conv2d(3, out_channels, (1, 1), bias=True)

        curr_dim = conv_dim
        self.rgb_to_features = ModuleList()

  
        # Discrimintor Conv Block
        def dis_initial_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))

        def dis_mid_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim+3, curr_dim, kernel_size=3, stride=1, padding=1,bias=False), nn.LeakyReLU(0.01),
                               nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))
        
        #def dis_mid_conv_block(curr_dim,K=4,S=2,P=1,bias_=False):
          #return nn.Sequential(nn.Conv2d(curr_dim+3, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               #nn.LeakyReLU(0.01))
          
        def dis_conv_block_without_rgb(curr_dim,K=4,S=2,P=1,bias_=False):
          return nn.Sequential(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=K, stride=S, padding=P,bias=False),
                               nn.LeakyReLU(0.01))

        # Initiate discrimator blocks module list
        self.Dis_blocks = ModuleList()

        # Fill the module list of both discriminator blocks and rgb converter
        for i in range(1, repeat_num):
          if i == 1: # first layer
            self.Dis_blocks.append(dis_initial_conv_block(curr_dim,4,2,1,False))
            from_rgb_ = from_rgb(int(curr_dim))
            self.rgb_to_features.append(from_rgb_)            
            curr_dim = curr_dim * 2
          elif i < (repeat_num-3) and (i != 1):
            self.Dis_blocks.append(dis_mid_conv_block(curr_dim,4,2,1,False))
            from_rgb_ = from_rgb(int(curr_dim))
            self.rgb_to_features.append(from_rgb_)            
            curr_dim = curr_dim * 2
          elif i == repeat_num: # last layer
            self.Dis_blocks.append(dis_conv_block_without_rgb(curr_dim,4,2,1,False))
          else:
            self.Dis_blocks.append(dis_conv_block_without_rgb(curr_dim,4,2,1,False))      
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num)) # this code was modified to fit kernel 8x8 to get the class
        print("kernel size",kernel_size)

        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        if image_size == 1024:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01),nn.Conv2d(c_dim, c_dim, kernel_size=kernel_size, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01),nn.Conv2d(c_dim, c_dim, kernel_size=kernel_size-2, stride=1, padding=0,bias=False), nn.LeakyReLU(0.01))
        elif image_size == 512:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size+2, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01)) 
        
        elif image_size == 256:
          self.conv2=nn.Sequential(nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size+1, stride=1, padding=0,bias=False),
                                   nn.LeakyReLU(0.01)) 

        
        
    def forward(self, inputs):
        #repeat_num=8;
        #depth=4

        # First Input
        y = self.rgb_to_features[0](inputs[0])
        #print("y rgb to features",y.size())
        y = self.Dis_blocks[0](y)
        #print("dis_block",y.size())

        #counter=0
          
        for x, block in \
                zip((inputs[1:]),
                    (self.Dis_blocks[1:])):
            
            #def get_shapes(Depth):
            #  return [inputs[i].size() for i in range(Depth)]
            #print('full input shape:', get_shapes(len(inputs)) )

            #print('x: ', x.size())
            #input_part = converter(x)  # convert the input:
            #print("input_part: ", input_part.size() )
            #print(converter)
            y = torch.cat((x, y), dim=1)  # concatenate the inputs:
            #print("concat input_part(here x instead) and y:",y.size())
            #print('block:', block)
            y = block(y)  # apply the block
            #print("y block(y):",y.size())
            #counter=counter+1
            #print(counter)

        y = self.Dis_blocks[-3](y)
        #print(y.size())
        y = self.Dis_blocks[-2](y)
        #print(y.size())
        y = self.Dis_blocks[-1](y)
        #print("y last",y.size())
        # calculate the final block:
        #input_part = self.final_converter(inputs[0])#
        #y = th.cat((input_part, y), dim=1)
        #y = self.final_block(y)

        out_src = self.conv1(y)
        #print("out_src: ",out_src.size())
        out_cls = self.conv2(y)
        #print("out_cls: ",out_cls.size())
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))



  
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class ResidualBlock(nn.Module):
    """"""Residual Block with instance normalization.""""""
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True), # nn.ReLU(inplace=True) originally
            nn.Conv2d(dim_out, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)


class Generator(nn.Module):
    """"""Generator network.""""""
    def __init__(self, ngpu=0, conv_dim=64, c_dim=5, repeat_num=6):
        super(Generator, self).__init__()
        self.ngpu = torch.cuda.device_count()
        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True)) # before it was True

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(2):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True)) # before it was True
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))

        # Up-sampling layers.
        for i in range(2):
            layers.append(nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True)) # was True
            curr_dim = curr_dim // 2

        layers.append(nn.Conv2d(curr_dim, 3, kernel_size=7, stride=1, padding=3, bias=False))
        self.main = nn.Sequential(*layers)

    def forward(self, x, c):
        # Replicate spatially and concatenate domain information.
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)
        return self.main(x)


class Discriminator(nn.Module):
    """"""Discriminator network with PatchGAN.""""""
    def __init__(self, ngpu=0, image_size=128, conv_dim=64, c_dim=5, repeat_num=6):
        super(Discriminator, self).__init__()
        self.ngpu=torch.cuda.device_count()
        layers = []
        layers.append(nn.Conv2d(3, conv_dim, kernel_size=4, stride=2, padding=1))
        layers.append(nn.LeakyReLU(0.01))
         

        curr_dim = conv_dim
        for i in range(1, repeat_num):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1))
            layers.append(nn.LeakyReLU(0.01))
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num))
        self.main = nn.Sequential(*layers)
        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, bias=False)
        
    def forward(self, x):
        h = self.main(x)
        out_src = self.conv1(h)
        out_cls = self.conv2(h)
        # h1 clone removed..probably not needed
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))
"""

# logger

import tensorflow as tf


class Logger(object):
    """Tensorboard logger."""

    def __init__(self, log_dir):
        """Initialize summary writer."""
        # self.writer = tf.summary.FileWriter(log_dir) # depreciated
        # self.writer = tf.compat.v1.summary.FileWriter(log_dir) # new working one
        
        #self.writer = tf.contrib.summary.FileWriter(log_dir)
        #self.writer = tf.train.SummaryWriter(log_dir)
        
        # for version 2.xx
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Add scalar summary."""
        #summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        #self.writer.add_summary(summary, step)
        
        # tf version 2.x
        with self.writer.as_default():
            tf.summary.scalar(tag, value, step=step)
            self.writer.flush()

            
# brats_auc

import numpy as np
from glob import glob
from PIL import Image
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

imgs = glob("brats_syn_256_lambda0.1/results/*.jpg")
gts = [0]*975 + [1]*486

preds = []

for i in range(len(imgs)):
    im = np.array(Image.open("brats_syn_256_lambda0.1/results/{}-images.jpg".format(i+1)))
    rows = np.split(im, im.shape[0]//256, axis=0)

    for r in rows:
        cols = np.split(r, 5, axis=1)
        preds.append(np.max( np.mean(cols[1], axis=-1) ))

#print(roc_auc_score(gts, preds))

            
# Solver

#from model import Generator
#from model import Discriminator
from torch.autograd import Variable
from torchvision.utils import save_image
import torch
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from sklearn.cluster import KMeans
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torch.nn.parallel import DataParallel


class Solver(object):
    """Solver for training and testing Fixed-Point GAN."""

    def __init__(self, data_loader, config):
        """Initialize configurations."""

        # Data loader.
        self.data_loader = data_loader

        # Model configurations.
        self.c_dim = config.c_dim
        self.c2_dim = config.c2_dim
        self.image_size = config.image_size
        self.g_conv_dim = config.g_conv_dim
        self.d_conv_dim = config.d_conv_dim
        self.g_repeat_num = config.g_repeat_num
        self.d_repeat_num = config.d_repeat_num
        self.lambda_cls = config.lambda_cls
        self.lambda_rec = config.lambda_rec
        self.lambda_gp = config.lambda_gp
        self.lambda_id = config.lambda_id
        self.depth = config.depth

        # Training configurations.
        self.dataset = config.dataset
        self.batch_size = config.batch_size
        self.num_iters = config.num_iters
        self.num_iters_decay = config.num_iters_decay
        self.g_lr = config.g_lr
        self.d_lr = config.d_lr
        self.n_critic = config.n_critic
        self.beta1 = config.beta1
        self.beta2 = config.beta2
        self.resume_iters = config.resume_iters
        self.selected_attrs = config.selected_attrs

        # Test configurations.
        self.test_iters = config.test_iters

        # Miscellaneous.
        self.use_tensorboard = config.use_tensorboard
        #self.ngpu= config.ngpu
        self.ngpu= torch.cuda.device_count()
        self.device = torch.device('cuda' if (torch.cuda.is_available() and self.ngpu>0) else 'cpu')
        print('Number of Cuda devices: ',torch.cuda.device_count())

        # Directories.
        self.log_dir = config.log_dir
        self.sample_dir = config.sample_dir
        self.model_save_dir = config.model_save_dir
        self.result_dir = config.result_dir

        # Step size.
        self.log_step = config.log_step
        self.sample_step = config.sample_step
        self.model_save_step = config.model_save_step
        self.lr_update_step = config.lr_update_step

        self.rank=0
        self.world_size=1

        # Build the model and tensorboard.
        self.build_model()
        if self.use_tensorboard:
          self.build_tensorboard()
    
    def setup(self,rank,world_size):
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        # initialize the process group
        dist.init_process_group("nccl", rank=rank, world_size=world_size)

    def build_model(self):
        """Create a generator and a discriminator."""

        if self.dataset in ['CelebA', 'BRATS', 'Directory']:
            self.G = Generator(self.ngpu,self.g_conv_dim, self.c_dim, self.g_repeat_num,self.depth)
            self.D = Discriminator(self.ngpu,self.image_size, self.d_conv_dim, self.c_dim, self.d_repeat_num,self.depth) 

        #self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
        #self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])
        self.print_network(self.G, 'Generator')
        self.print_network(self.D, 'Discriminator')
    
        self.G.to(self.device)
        self.D.to(self.device)

        # Handle multi-gpu if desired
        if (self.device.type == 'cuda') and (self.ngpu > 1):
          # added for parallelism
          #for i in list(range(self.ngpu)):
          #  torch.cuda.set_device(i)       
          #  self.setup(self.rank,self.world_size)
          #  self.G = DistributedDataParallel(self.G, device_ids=[i],output_device=i)
          #  self.D = DistributedDataParallel(self.D, device_ids=[i],output_device=i)

          #self.setup(self.rank,self.world_size)
          #self.G = DistributedDataParallel(self.G, device_ids=list(range(self.ngpu)))
          #self.D = DistributedDataParallel(self.D, device_ids=list(range(self.ngpu)))
          
          self.G = DataParallel(self.G, device_ids=list(range(self.ngpu)))
          self.D = DataParallel(self.D, device_ids=list(range(self.ngpu)))

          self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
          self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])

        else:

          self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
          self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])

    def recreate_image(self, codebook, labels, w, h):
        """Recreate the (compressed) image from the code book & labels"""
        d = codebook.shape[1]
        image = np.zeros((w, h, d))
        label_idx = 0
        for i in range(w):
            for j in range(h):
                image[i][j] = codebook[labels[label_idx]]
                # orginal
                # label_idx += 1
                label_idx = label_idx + 1
        return image

    def print_network(self, model, name):
        """Print out the network information."""
        num_params = 0
        for p in model.parameters():
            # original
            # num_params += p.numel()
            num_params = num_params + p.numel()
        print(model)
        print(name)
        print("The number of parameters: {}".format(num_params))

    def restore_model(self, resume_iters):
        """Restore the trained generator and discriminator."""
        print('Loading the trained models from step {}...'.format(resume_iters))
        G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(resume_iters))
        D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(resume_iters))
        self.G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage)) # Load all tensors onto the CPU, using a function
        self.D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage)) # Load all tensors onto the CPU, using a function # to avoid GPU RAM surge when loading a model checkpoint.

    def build_tensorboard(self):
        """Build a tensorboard logger."""
        #from logger import Logger
        self.logger = Logger(self.log_dir)

    def update_lr(self, g_lr, d_lr):
        """Decay learning rates of the generator and discriminator."""
        for param_group in self.g_optimizer.param_groups:
            param_group['lr'] = g_lr
        for param_group in self.d_optimizer.param_groups:
            param_group['lr'] = d_lr

    def reset_grad(self):
        """Reset the gradient buffers."""
        self.g_optimizer.zero_grad()
        self.d_optimizer.zero_grad()

    def denorm(self, x):
        """Convert the range from [-1, 1] to [0, 1]."""
        out = (x + 1) / 2
        return out.clamp_(0, 1)

    def gradient_penalty(self, y, x):
        """Compute gradient penalty: (L2_norm(dy/dx) - 1)**2."""
        weight = torch.ones(y.size()).to(self.device)
        dydx = torch.autograd.grad(outputs=y,
                                   inputs=x,
                                   grad_outputs=weight,
                                   retain_graph=True,
                                   create_graph=True,
                                   only_inputs=True)[0]

        dydx = dydx.view(dydx.size(0), -1)
        dydx_l2norm = torch.sqrt(torch.sum(dydx**2, dim=1))
        return torch.mean((dydx_l2norm-1)**2)

    def label2onehot(self, labels, dim):
        """Convert label indices to one-hot vectors."""
        batch_size = labels.size(0)
        out = torch.zeros(batch_size, dim)
        out[np.arange(batch_size), labels.long()] = 1
        return out

    def create_labels(self, c_org, c_dim=5, dataset='CelebA', selected_attrs=None):
        """Generate target domain labels for debugging and testing."""
        # Get hair color indices.
        if dataset in ['CelebA']:
            hair_color_indices = []
            for i, attr_name in enumerate(selected_attrs):
                if attr_name in ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']:
                    hair_color_indices.append(i)

        c_trg_list = []
        for i in range(c_dim):
            if dataset in ['CelebA']:
                c_trg = c_org.clone()
                if i in hair_color_indices:  # Set one hair color to 1 and the rest to 0.
                    c_trg[:, i] = 1
                    for j in hair_color_indices:
                        if j != i:
                            c_trg[:, j] = 0
                else:
                    c_trg[:, i] = (c_trg[:, i] == 0)  # Reverse attribute value.
            elif dataset == 'BRATS':
                c_trg = c_org.clone()
                c_trg[:, i] = (c_trg[:, i] == 0)  # Reverse attribute value.
            elif dataset == 'Directory':
                c_trg = self.label2onehot(torch.ones(c_org.size(0))*i, c_dim)

            c_trg_list.append(c_trg.to(self.device))
        return c_trg_list

    def classification_loss(self, logit, target, dataset='CelebA'):
        """Compute binary or softmax cross entropy loss."""
        if dataset in ['CelebA', 'BRATS']:
            return F.binary_cross_entropy_with_logits(logit, target, reduction='sum') / logit.size(0)  # size_average=False inside binary_cross_entropy_with_logits
        elif dataset == 'Directory':
            return F.cross_entropy(logit, target)

    def create_grid(self, samples, img_files):
        """
        utility function to create a grid of GAN samples
        :param samples: generated samples for storing list[Tensors]
        :param img_files: list of names of files to write
        :return: None (saves multiple files)
        """
        from torchvision.utils import save_image
        from torch.nn.functional import interpolate
        from numpy import sqrt, power

        # dynamically adjust the colour of the images
        samples = [Generator.adjust_dynamic_range(sample) for sample in samples]

        # resize the samples to have same resolution:
        for i in range(len(samples)):
            samples[i] = interpolate(samples[i],
                                     scale_factor=power(2,
                                                        self.depth - 1 - i))
        # save the images:
        for sample, img_file in zip(samples, img_files):
            save_image(sample, img_file, nrow=int(sqrt(sample.shape[0])),
                       normalize=True, scale_each=True, padding=0) 

    def train(self):
        """Train Fixed-Point GAN within a single dataset."""
        # Set data loader.
        if self.dataset in ['CelebA', 'BRATS', 'Directory']:
            data_loader = self.data_loader

        # Fetch fixed inputs for debugging.
        data_iter = iter(data_loader)
        x_fixed, c_org = next(data_iter)
        x_fixed = x_fixed.to(self.device)
        c_fixed_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)
        #print('c_fixed_list',c_fixed_list)

        # Learning rate cache for decaying.
        g_lr = self.g_lr
        d_lr = self.d_lr

        # Start training from scratch or resume training.
        start_iters = 0
        if self.resume_iters:
            start_iters = self.resume_iters
            self.restore_model(self.resume_iters)

        # Start training.
        print('Start training...')
        start_time = time.time()
        for i in range(start_iters, self.num_iters):

            # =================================================================================== #
            #                             1. Preprocess input data                                #
            # =================================================================================== #
            

            # Fetch real images and labels.
            try:
                x_real, label_org = next(data_iter)
                #print("x_real",x_real.size()) # added to test the dimensions
                #print("label_org",label_org.size()) # added to test the dimensions
            except:
                data_iter = iter(data_loader)
                x_real, label_org = next(data_iter)

                #print("x_real: ",x_real.size()) # added to test the dimensions
                #print("label_org: ",label_org.size()) # added to test the dimensions

            # Generate target domain labels randomly.
            rand_idx = torch.randperm(label_org.size(0))
            label_trg = label_org[rand_idx]

            #print("rand_idx",rand_idx.size()) # added to test the dimensions
            #print("label_trg",label_trg.size()) # added to test the dimensions

            if self.dataset in ['CelebA', 'BRATS']:
                c_org = label_org.clone()
                c_trg = label_trg.clone()
            elif self.dataset == 'Directory':
                c_org = self.label2onehot(label_org, self.c_dim)
                c_trg = self.label2onehot(label_trg, self.c_dim)
            
            # added to output everything on 1 gpu 
            if (self.device.type == 'cuda') and (self.ngpu > 1):
              torch.cuda.set_device(0)   
            
            x_real = x_real.to(self.device)           # Input images.
            c_org = c_org.to(self.device)             # Original domain labels.
            c_trg = c_trg.to(self.device)             # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            #print("c_org",c_org.size()) # added to test the dimensions
            #print("c_targ",c_trg.size()) # added to test the dimensions
            

            # testing batch_size and image pyramid
            from torch.nn.functional import avg_pool2d
            batch_images=x_real  
            batch_images = [batch_images] + [avg_pool2d(batch_images, int(np.power(2, i)))
                                     for i in range(1, self.depth)]
                            
            #print('image length list: ',len(batch_images)) # added : equal number of depth or layers
            #def get_shapes(Depth):
            #  return [batch_images[i].size() for i in range(Depth)]
            #print('images shape:', get_shapes(self.depth) )


            # list needs to be added to a torch to be able to be used with to(self.device)
            #x_real=[_x_real.to(self.device) for _x_real in batch_images]    # input images
            #data = [_data.cuda() for _data in data] 
            #c_org = c_org.repeat(5,1,1).to(self.device)           # Original domain labels.
            #c_trg = c_trg.repeat(5,1,1).to(self.device)         # Target domain labels.
            #label_org = label_org.repeat(5,1,1).to(self.device)     # Labels for computing classification loss.
            #label_trg = label_trg.repeat(5,1,1).to(self.device)     # Labels for computing classification loss.

            #for _x_real in batch_images: 
            """
            def padding(ListofTensorImages):
              out=[]
              for img in ListofTensorImages:
                print(img.size())
                equ= (1024 - img.size(-1))/2
                print(equ)
                m = nn.ZeroPad2d(int(equ))
                out.append(m(img))
              return out

            ImagesWithPadding=padding(batch_images)
            for i in range(len(ImagesWithPadding)):
              print(ImagesWithPadding[i].size())

            stackedImages= torch.stack([ImagesWithPadding[0],ImagesWithPadding[1],ImagesWithPadding[2],ImagesWithPadding[3],ImagesWithPadding[4]],dim=0)
            """

            x_real_d=batch_images #.to(self.device)
            c_org = c_org.to(self.device)           # Original domain labels.
            c_trg = c_trg.to(self.device)         # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            #def get_shapes_01(Depth):
            #  return [batch_images[i].size() for i in range(Depth)],[c_org[i].size() for i in range(Depth)],[c_trg[i].size() for i in range(Depth)],[label_org[i].size() for i in range(Depth)],[label_trg[i].size() for i in range(Depth)]
            #print('batch_images, c_org, c_trig , label_org , label_trg :', get_shapes_01(depth__) )  

   
            # =================================================================================== #
            #                             2. Train the discriminator                              #
            # =================================================================================== #


          #  # Compute loss with real images.
            
            t0=time.time()
            out_src, out_cls = self.D(x_real_d)     
            #print("out_src",out_src.size()) # added to test the dimensions
            #print("out_cls",out_cls.size()) # added to test the dimensions       
            d_loss_real = - torch.mean(out_src)
            d_loss_cls = self.classification_loss(out_cls, label_org, self.dataset)
            #print("Loss with real images",time.time()-t0)

            #print("out_src",out_src.size()) # added to test the dimensions
            #print("out_cls",out_cls.size()) # added to test the dimensions
            #print("d_loss_real",d_loss_real.size()) # added to test the dimensions
            #print("d_loss_cls",d_loss_cls.size()) # added to test the dimensions
          
          # Compute loss with fake images.
            t0=time.time()
            delta = self.G(x_real, c_trg) # c_trg is target class  # output 5 images : output the difference   
            
            #def get_shapes(Depth):
            #  return [delta[i].size() for i in range(Depth)]
            #print('images shape delta, results of Generator:', get_shapes(depth__) )
            #print("delta last batch",delta[-1].size())

            delta=delta[::-1] # inverse List
            x_fake=[torch.tanh(x_real_d[i] + delta[i]) for i in range(len(x_real_d))] # of 1 image 4x3x1024x1024
            x_fake=[x_fake[i].detach() for i in range(len(x_real_d))]
            out_src, out_cls = self.D(x_fake)
            d_loss_fake = torch.mean(out_src)
            #print("Loss with fake images",time.time()-t0)
            
            #print("out_src",out_src.size()) # added to test the dimensions
            #print("out_cls",out_cls.size()) # added to test the dimensions
            #print("d_loss_fake",d_loss_fake.size()) # added to test the dimensions
  

          # Compute loss for gradient penalty.
            t0=time.time()
            alpha = torch.rand(x_real.size(0), 1, 1, 1).to(self.device) # x_real.size(0) is the nb of batch
            #x_hat = (alpha * x_real.data + (1 - alpha) * x_fake.data).requires_grad_(True) 
            x_hat = [(alpha * x_real_d[i].data + (1 - alpha) * x_fake[i].data).requires_grad_(True) for i in range(len(x_real_d))]
            out_src, _ = self.D(x_hat)
            d_loss_gp = self.gradient_penalty(out_src, x_hat)
            #print("Loss for gredient penalty",time.time()-t0)
            
            #check the gpu before changing
            #print(d_loss_gp.get_device())

            #print("alpha, compute gradient penalty",alpha) # added to test the dimensions
            #print("out_src",out_src.size()) # added to test the dimensions
            #print("d_loss_gp",d_loss_gp) # added to test the dimensions
    

            # Backward and optimize.#
            d_loss = d_loss_real + d_loss_fake + self.lambda_cls * d_loss_cls + self.lambda_gp * d_loss_gp 

            #print("d_loss",d_loss) # added to test the dimensions
            
            t0=time.time()
            self.reset_grad()

            if (self.device.type == 'cuda') and (self.ngpu > 1):
              d_loss.mean().backward()
            else:
              d_loss.backward()

            self.d_optimizer.step()
            #print("back propagation and optimization",time.time()-t0)
            # Logging.
            loss = {}
            loss['D/loss_real'] = d_loss_real.item()
            loss['D/loss_fake'] = d_loss_fake.item()
            loss['D/loss_cls'] = d_loss_cls.item()
            loss['D/loss_gp'] = d_loss_gp.item()
            # added
            loss['d_loss'] = d_loss.item()
            
            # =================================================================================== #
            #                               3. Train the generator                                #
            # =================================================================================== #
            
            if (i+1) % self.n_critic == 0:

                # Original-to-target domain.
                t0=time.time()
                delta = self.G(x_real, c_trg)
                delta=delta[::-1]
                x_fake=[torch.tanh(x_real_d[i] + delta[i]) for i in range(len(x_real_d))] # of 1 image 4x3x1024x1024
                #x_fake=[x_fake[i].detach() for i in range(len(x_real_d))]
                out_src, out_cls = self.D(x_fake)
                #x_fake = torch.tanh(x_real_d + delta)
                #out_src, out_cls = self.D(x_fake)
                g_loss_fake = - torch.mean(out_src)
                g_loss_cls = self.classification_loss(out_cls, label_trg, self.dataset)
                
                #print("Original-to-target domain : Generator",time.time()-t0)

                #print("delta, generator",delta.size()) # added to test the dimensions
                #print("x_fake: tanh(x_real + delta)",x_fake.size()) # added to test the dimensions
                #print("out_src",out_src.size()) # added to test the dimensions
                #print("out_cls",out_cls.size()) # added to test the dimensions
                #print("g_loss_fake",g_loss_fake) # added to test the dimensions
                #print("g_loss_cls",g_loss_cls) # added to test the dimensions



                # Original-to-original domain.
                t0=time.time()

                delta_id = self.G(x_real, c_org)
                delta_id=delta_id[::-1]
                #x_fake_id = torch.tanh(x_real_d + delta_id)
                x_fake_id=[torch.tanh(x_real_d[i] + delta_id[i]) for i in range(len(x_real_d))]
                out_src_id, out_cls_id = self.D(x_fake_id)
                g_loss_fake_id = - torch.mean(out_src_id)
                g_loss_cls_id = self.classification_loss(out_cls_id, label_org, self.dataset)
                #g_loss_id = torch.mean(torch.abs(x_real - torch.tanh(delta_id + x_real)))

                
                def image_padding(ListofTensorImages):
                  out=[]
                  for img in ListofTensorImages:
                    #print(img.size())
                    equ= (1024 - img.size(-1))/2
                    #print(equ)
                    m = nn.ZeroPad2d(int(equ))
                    out.append(m(img))
                  return out

                g_loss_id = torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_fake_id[i]) for i in range(len(x_real_d))])))
                #print(g_loss_id)
                #print("Original-to-original domain : Generator",time.time()-t0)

                #print("delta_id, generator",delta_id.size()) # added to test the dimensions
                #print("x_fake_id: tanh(x_real + delta_id)",x_fake_id.size()) # added to test the dimensions
                #print("out_src_id",out_src_id.size()) # added to test the dimensions
                #print("out_cls_id",out_cls_id.size()) # added to test the dimensions
                #print("g_loss_fake_id",g_loss_fake_id) # added to test the dimensions
                #print("g_loss_cls_id",g_loss_cls_id) # added to test the dimensions
                #print("g_loss_id",g_loss_id) # added to test the dimensions


                # Target-to-original domain.
                t0=time.time()

                #print("x_fake[-1] size: ",x_fake[0].size()) # added to test the dimensions
                delta_reconst = self.G(x_fake[0], c_org)
                delta_reconst=delta_reconst[::-1]
                #print("delta_reconst[-1] size: ",delta_reconst[-1].size()) # added to test the dimensions


                x_reconst=[torch.tanh(x_fake[i] + delta_reconst[i]) for i in range(len(x_real_d))]
                #x_reconst = torch.tanh(x_fake + delta_reconst)
                #g_loss_rec = torch.mean(torch.abs(x_real - x_reconst))
                g_loss_rec= torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_reconst[i]) for i in range(len(x_real_d))])))

                #print("delta_reconst, generator",delta_reconst.size()) # added to test the dimensions
                #print("x_reconst: tanh(x_real + delta_reconst)",x_reconst.size()) # added to test the dimensions
                #print("g_loss_rec",g_loss_rec) # added to test the dimensions

                #print("Target-to-original domain : Generator",time.time()-t0)

                # Original-to-original domain.
                t0=time.time()

                delta_reconst_id = self.G(x_fake_id[0], c_org) 
                delta_reconst_id = delta_reconst_id[::-1]
                x_reconst_id=[torch.tanh(x_fake_id[i] + delta_reconst_id[i]) for i in range(len(x_real_d))]
                #x_reconst_id = torch.tanh(x_fake_id + delta_reconst_id)
                #g_loss_rec_id = torch.mean(torch.abs(x_real - x_reconst_id))
                g_loss_rec_id= torch.mean(torch.stack(image_padding([torch.abs(x_real_d[i] - x_reconst_id[i]) for i in range(len(x_real_d))])))  # 5 scales images 


                #print("delta_reconst_id, generator",delta_reconst_id.size()) # added to test the dimensions
                #print("x_reconst_id: tanh(x_real + delta_reconst_id)",x_reconst_id.size()) # added to test the dimensions
                #print("g_loss_rec_id",g_loss_rec_id) # added to test the dimensions

                #print("Original-to-original domain : Generator",time.time()-t0)


                # Backward and optimize.
                g_loss_same = g_loss_fake_id + self.lambda_rec * g_loss_rec_id + self.lambda_cls * g_loss_cls_id + self.lambda_id * g_loss_id
                g_loss = g_loss_fake + self.lambda_rec * g_loss_rec + self.lambda_cls * g_loss_cls + g_loss_same

                #print("g_loss_same",g_loss_same.size()) # added to test the dimensions
                #print("g_loss",g_loss) # added to test the dimensions

                t0=time.time()
                self.reset_grad()
                #g_loss.backward()

                if (self.device.type == 'cuda') and (self.ngpu > 1):
                  g_loss.mean().backward()
                else:
                  g_loss.backward()

                self.g_optimizer.step()
                #print("back Propagation: Generator",time.time()-t0)

                # Logging.
                loss['G/loss_fake'] = g_loss_fake.item()
                loss['G/loss_rec'] = g_loss_rec.item()
                loss['G/loss_cls'] = g_loss_cls.item()
                loss['G/loss_fake_id'] = g_loss_fake_id.item()
                loss['G/loss_rec_id'] = g_loss_rec_id.item()
                loss['G/loss_cls_id'] = g_loss_cls_id.item()
                loss['G/loss_id'] = g_loss_id.item()
                # added
                loss['g_loss'] = g_loss.item()

            # =================================================================================== #
            #                                 4. Miscellaneous                                    #
            # =================================================================================== #

            # Print out training information.
            if (i+1) % self.log_step == 0:
                et = time.time() - start_time
                et = str(datetime.timedelta(seconds=et))[:-7]
                log = "Elapsed [{}], Iteration [{}/{}]".format(et, i+1, self.num_iters)
                for tag, value in loss.items():
                    log += ", {}: {:.4f}".format(tag, value)
                print(log,flush=True)

                if self.use_tensorboard:
                    for tag, value in loss.items():
                        self.logger.scalar_summary(tag, value, i+1)

            # Translate fixed images for debugging.
            if (i+1) % self.sample_step == 0:
              from torch.nn.functional import avg_pool2d
              #batch_images_fixed=x_fixed  
              #depth__=5
              #batch_images_fixed = [batch_images_fixed] + [avg_pool2d(batch_images_fixed, int(np.power(2, i)))
              #                       for i in range(1, depth__)]
              with torch.no_grad():
                  x_fake_list = [x_fixed]
                  print("fake_list[0]",x_fake_list[0].size()) # 3x3x1024x1024   
                  for c_fixed in c_fixed_list:
                      delta = self.G(x_fixed, c_fixed)
                      delta = delta[::-1]
                      gen_img=torch.tanh(delta[0] + x_fixed)
                      x_fake_list.append(gen_img) # add to list
                  x_concat = torch.cat(x_fake_list, dim=3)
                  sample_path = os.path.join(self.sample_dir, '{}-images.jpg'.format(i+1))
                  save_image(self.denorm(x_concat.data.cpu()), sample_path, nrow=1, padding=0)
                  print('Saved real and fake images into {}...'.format(sample_path))


                  # create a grid of samples and save it
                  #reses = [str(int(np.power(2, dep+4))) + "_x_"
                  #+ str(int(np.power(2, dep+4)))
                  #for dep in reversed(range(2,depth__ + 2))]

                  #gen_img_files = [os.path.join(self.sample_dir, res, "gen_" +
                  #                              str(i) + ".png")
                  #                  for res in reses]

                  # Make sure all the required directories exist
                  # otherwise make them
                  #os.makedirs(self.sample_dir, exist_ok=True)
                  #for gen_img_file in gen_img_files:
                  #    os.makedirs(os.path.dirname(gen_img_file), exist_ok=True)

                  # fake list should contain the x_fake_list of different versions
                  #print("x_fixed",x_fixed[0].size()) 
                  #print("delta",delta[0].size())      
                  #gen_imgs=[torch.tanh(batch_images_fixed[i] + delta[i]) for i in range(len(x_real_d))]
                  #self.create_grid(gen_imgs,gen_img_files)

            # Save model checkpoints.
            if (i+1) % self.model_save_step == 0:
                G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(i+1))
                D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(i+1))
                
                if (self.device.type == 'cuda') and (self.ngpu > 1):
                  torch.save(self.G.module.state_dict(), G_path) # loading doesn't change
                  torch.save(self.D.module.state_dict(), D_path) # loading doesn't change
                else:
                  torch.save(self.G.state_dict(), G_path)
                  torch.save(self.D.state_dict(), D_path)

                print('Saved model checkpoints into {}...'.format(self.model_save_dir))

            # Decay learning rates.
            if (i+1) % self.lr_update_step == 0 and (i+1) > (self.num_iters - self.num_iters_decay):
                g_lr = g_lr - (self.g_lr / float(self.num_iters_decay))
                d_lr = d_lr - (self.d_lr / float(self.num_iters_decay))
                #original
                #g_lr -= (self.g_lr / float(self.num_iters_decay))
                #d_lr -= (self.d_lr / float(self.num_iters_decay))
                self.update_lr(g_lr, d_lr)
                print ('Decayed learning rates, g_lr: {}, d_lr: {}.'.format(g_lr, d_lr))


    def test(self):
        """Translate images using Fixed-Point GAN trained on a single dataset."""
        # Load the trained generator.
        self.restore_model(self.test_iters)
        
        # Set data loader.
        if self.dataset in ['CelebA', 'Directory']:
            data_loader = self.data_loader
        
        with torch.no_grad():
            for i, (x_real, c_org) in enumerate(data_loader):

                # Prepare input images and target domain labels.
                x_real = x_real.to(self.device)
                c_trg_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)

                # Translate images.
                x_fake_list = [x_real]
                for c_trg in c_trg_list:
                    x_fake_list.append(torch.tanh(x_real + self.G(x_real, c_trg)))

                # Save the translated images.
                x_concat = torch.cat(x_fake_list, dim=3)
                result_path = os.path.join(self.result_dir, '{}-images.jpg'.format(i+1))
                save_image(self.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
                print('Saved real and fake images into {}...'.format(result_path))



    def test_brats(self):
        """Translate images using Fixed-Point GAN trained on a single dataset."""
        # Load the trained generator.
        self.restore_model(self.test_iters)
        
        # Set data loader.
        if self.dataset in ['BRATS']:
            data_loader = self.data_loader
        
        with torch.no_grad():
            for i, (x_real, c_org) in enumerate(data_loader):
                x_real = x_real.to(self.device)

                c_trg = c_org.clone()
                c_trg[:, 0] = 0 # always to healthy              
                c_trg_list = [c_trg.to(self.device)]

                # Translate images.
                x_fake_list = [x_real]
                for c_trg in c_trg_list:
                    delta = self.G(x_real, c_trg)
                    delta_org = torch.abs(torch.tanh(delta + x_real) - x_real) - 1.0
                    delta_gray = np.mean(delta_org.data.cpu().numpy(), axis=1)
                    delta_gray_norm = []

                    loc = []
                    cls_mul = []

                    for indx in range(delta_gray.shape[0]):
                        temp = delta_gray[indx, :, :] + 1.0  
                        tempimg_th = np.percentile(temp, 99)
                        tempimg = np.float32(temp >= tempimg_th)
                        temploc = np.reshape(tempimg, (self.image_size*self.image_size, 1))

                        kmeans = KMeans(n_clusters=2, random_state=0).fit(temploc)
                        labels = kmeans.predict(temploc)

                        recreated_loc = self.recreate_image(kmeans.cluster_centers_, labels, self.image_size, self.image_size)
                        recreated_loc = ((recreated_loc - np.min(recreated_loc)) / (np.max(recreated_loc) - np.min(recreated_loc)))

                        loc.append(recreated_loc)
                        delta_gray_norm.append( tempimg )


                    loc = np.array(loc, dtype=np.float32)[:, :, :, 0]
                    delta_gray_norm = np.array(delta_gray_norm)

                    loc = (loc * 2.0) - 1.0
                    delta_gray_norm = (delta_gray_norm * 2.0) - 1.0

                    x_fake_list.append( torch.from_numpy(np.repeat(delta_gray[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # difference map
                    x_fake_list.append( torch.from_numpy(np.repeat(delta_gray_norm[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # localization thershold
                    x_fake_list.append( torch.from_numpy(np.repeat(loc[:, np.newaxis, :, :], 3, axis=1)).to(self.device) ) # localization kmeans
                    x_fake_list.append( torch.tanh(delta + x_real) ) # generated image

                # Save the translated images.
                x_concat = torch.cat(x_fake_list, dim=3)
                result_path = os.path.join(self.result_dir, '{}-images.jpg'.format(i+1))
                save_image(self.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
                print('Saved real and fake images into {}...'.format(result_path))



    def cleanup():
      dist.destroy_process_group()   

# Solver               

from torch.utils import data
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from PIL import Image
import torch
import os
import random
from glob import glob


class CelebA(data.Dataset):
    """Dataset class for the CelebA dataset."""

    def __init__(self, image_dir, attr_path, selected_attrs, transform, mode):
        """Initialize and preprocess the CelebA dataset."""
        self.image_dir = image_dir
        self.attr_path = attr_path
        self.selected_attrs = selected_attrs
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.test_dataset = []
        self.attr2idx = {}
        self.idx2attr = {}
        self.preprocess()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        else:
            self.num_images = len(self.test_dataset)

    def preprocess(self):
        """Preprocess the CelebA attribute file."""
        lines = [line.rstrip() for line in open(self.attr_path, 'r')]
        all_attr_names = lines[1].split()
        for i, attr_name in enumerate(all_attr_names):
            self.attr2idx[attr_name] = i
            self.idx2attr[i] = attr_name

        lines = lines[2:]
        random.seed(1234)
        random.shuffle(lines)
        for i, line in enumerate(lines):
            split = line.split()
            filename = split[0]
            values = split[1:]

            label = []
            for attr_name in self.selected_attrs:
                idx = self.attr2idx[attr_name]
                label.append(values[idx] == '1')

            if (i+1) < 2000:
                self.test_dataset.append([filename, label])
            else:
                self.train_dataset.append([filename, label])

        print('Finished preprocessing the CelebA dataset...')

    def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.test_dataset
        filename, label = dataset[index]

        # when gcp is used 
        os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
        Bucket_name=config.GCP_Bucket_name
        # Use Key authentication
        storage_client = storage.Client()
        # create bucket
        bucket = storage_client.get_bucket(Bucket_name)

        # 'data/celeba/images/'
        # data_hd/CelebA/
        blob = bucket.blob('data_hd/CelebA/'+filename)
        filename__=blob.download_as_string()
        image=Image.open(BytesIO(filename__))

        # when gcp is not used
        #image = Image.open(os.path.join(self.image_dir, filename))
        return self.transform(image), torch.FloatTensor(label)

    def __len__(self):
        """Return the number of images."""
        return self.num_images


class BRATS_SYN(data.Dataset):
    """Dataset class for the BRATS dataset."""

    def __init__(self, image_dir, transform, mode):
        """Initialize and Load the BRATS dataset."""
        self.image_dir = image_dir
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.test_dataset = []
        self.load_data()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        else:
            self.num_images = len(self.test_dataset)

    def load_data(self):
        """Load BRATS dataset"""
        
        # Load test dataset
        test_neg = glob(os.path.join(self.image_dir, 'test', 'negative', '*jpg'))
        test_pos = glob(os.path.join(self.image_dir, 'test', 'positive', '*jpg'))

        for filename in test_neg:
            self.test_dataset.append([filename, [0]])

        for filename in test_pos:
            self.test_dataset.append([filename, [1]])


        # Load train dataset
        train_neg = glob(os.path.join(self.image_dir, 'train', 'negative', '*jpg'))
        train_pos = glob(os.path.join(self.image_dir, 'train', 'positive', '*jpg'))

        for filename in train_neg:
            self.train_dataset.append([filename, [0]])

        for filename in train_pos:
            self.train_dataset.append([filename, [1]])

        print('Finished loading the BRATS dataset...')

    def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.test_dataset
        filename, label = dataset[index]
        image = Image.open(filename)
        return self.transform(image), torch.FloatTensor(label)

    def __len__(self):
        """Return the number of images."""
        return self.num_images


def get_loader(image_dir, attr_path, selected_attrs, crop_size=178, image_size=128, 
               batch_size=16, dataset='CelebA', mode='train', num_workers=1):
    """Build and return a data loader."""
    transform = []
    if mode == 'train':
        transform.append(T.RandomHorizontalFlip())
    transform.append(T.CenterCrop(crop_size))
    transform.append(T.Resize(image_size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    if dataset == 'CelebA':
        dataset =  CelebA(image_dir, attr_path, selected_attrs, transform, mode)
    elif dataset == 'BRATS':
        dataset = BRATS_SYN(image_dir, transform, mode)
    elif dataset == 'Directory':
        dataset = ImageFolder(image_dir, transform)

    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
    return data_loader

                                
                
# Main

import os
import argparse
#from solver import Solver
#from data_loader import get_loader
from torch.backends import cudnn
from google.cloud import storage
import tensorflow as tf
import time
from io import BytesIO
#from PIL import Image
import zipfile
from glob import glob
from tensorboard import version

# detect directory
# current directory: path at terminal when executing this file
cwd = os.getcwd()
print("Curret Directory: ",cwd)


def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training.
    cudnn.benchmark = True #True
    cudnn.enabled = True # added
    torch.cuda.empty_cache()
    

    # Create directories if not exist.
    if not os.path.exists(config.log_dir):
        os.makedirs(config.log_dir)
    print(os.path.exists(config.log_dir))
    if not os.path.exists(config.model_save_dir):
        os.makedirs(config.model_save_dir)
    print(os.path.exists(config.model_save_dir))
    if not os.path.exists(config.sample_dir):
        os.makedirs(config.sample_dir)
    print(os.path.exists(config.sample_dir))
    if not os.path.exists(config.result_dir):
        os.makedirs(config.result_dir)
    print(os.path.exists(config.result_dir))

    #os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
    Bucket_name=config.GCP_Bucket_name

    # Use Key authentication
    #storage_client = storage.Client()

    # create bucket
    #bucket = storage_client.get_bucket(Bucket_name)

    # read dataset.zip from bucket and copy it to directory
    #t0=time.time()
    #for blob in bucket.list_blobs(prefix= config.dataset_in_cloud):
    #  print(blob.name)
    #  tempfile= cwd + "/dataset__gcp.zip"
    #  blob.download_to_filename(tempfile)
    #t_end_zip=time.time()-t0
    #print('dataset gcp moved to local')

    #time.sleep(1)

    # read from directort zip file and extract
    #t0=time.time()
    #with zipfile.ZipFile("dataset__gcp.zip","r") as zip_ref:
    #    zip_ref.extractall(config.dataset_folder_name)
    #t_end_unzip=time.time()-t0
    #print('Extraction completed')

    #time.sleep(1)


        
    # Data loader.
    data_loader = None

    # 'data/celeba/images'
    # config.image_dir

    if config.dataset in ['CelebA']:#CelebA
        data_loader = get_loader(config.image_dir, config.attr_path, config.selected_attrs,
                                   config.crop_size, config.image_size, config.batch_size,
                                   'CelebA', config.mode, config.num_workers)


    elif config.dataset in ['BRATS']:
        data_loader = get_loader('{}/brats/syn'.format(config.dataset_folder_name), None, None,
                                   config.crop_size, config.image_size, config.batch_size,
                                   'BRATS', config.mode, config.num_workers)


    elif config.dataset in ['Directory']:
        data_loader = get_loader('config.image_dir', None, None,
                                 config.crop_size, config.image_size, config.batch_size,
                                 'Directory', config.mode, config.num_workers)

        
    # Solver for training and testing Fixed-Point GAN.
    solver = Solver(data_loader, config)

    if config.mode == 'train':
        if config.dataset in ['CelebA', 'BRATS', 'Directory']:  #CelebA
            solver.train()
    elif config.mode == 'test':
        if config.dataset in ['CelebA', 'Directory']:
            solver.test()
    elif config.mode == 'test_brats':
        if config.dataset in ['BRATS']:
            solver.test_brats()

    def Copy_local_dir_to_GCP_bucket(directory,folder_name_to_be_copied,gcp_output_folder):
        
        # to be later on fixed to be controlled from the main
        os.environ['GCLOUD_PROJECT'] = config.GCP_project_id
        Bucket_name=config.GCP_Bucket_name
        # Use Key authentication
        storage_client = storage.Client()
        # create bucket
        bucket = storage_client.get_bucket(Bucket_name)

        # Get the list of all files in directory tree at given path
        path = directory + "/" + folder_name_to_be_copied + "/"
        #we shall store all the file names in this list
        filelist = []

        for root, dirs, files in os.walk(path):
          for file in files:
            #append the file name to the list
            filelist.append(os.path.join(root,file))

        # write files and subdirectories to gcp
        for name in filelist:
            # print(name)
            blob=bucket.blob(name.replace(directory+"/" , gcp_output_folder+"/"))
            blob.upload_from_filename(name)
        return print('Output were transfered to gcp bucket')


    if config.dataset == 'BRATS':
      t0=time.time()
      Copy_local_dir_to_GCP_bucket(cwd,'brats_syn_256_lambda0.1', bucket,"Output")
      t_end_copy_folder_to_gcp=time.time()-t0
      print('time to end copying folders to gcp bucket',t_end_copy_folder_to_gcp)
    elif config.dataset == 'CelebA': # CelebA
      t0=time.time()
      Copy_local_dir_to_GCP_bucket(cwd,'celeba_1',"Output")
      t_end_copy_folder_to_gcp=time.time()-t0
      print('time to end copying folders to gcp bucket',t_end_copy_folder_to_gcp)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Model configuration.
    parser.add_argument('--c_dim', type=int, default=5, help='dimension of domain labels (1st dataset)')
    parser.add_argument('--c2_dim', type=int, default=8, help='dimension of domain labels (2nd dataset)')
    parser.add_argument('--crop_size', type=int, default=1024, help='crop size for the images') # was 178
    parser.add_argument('--image_size', type=int, default=128, help='image resolution')
    parser.add_argument('--g_conv_dim', type=int, default=16, help='number of conv filters in the first layer of G') # was 64
    parser.add_argument('--d_conv_dim', type=int, default=16, help='number of conv filters in the first layer of D') # was 64
    parser.add_argument('--g_repeat_num', type=int, default=8, help='number of residual blocks in G') # ws 6
    parser.add_argument('--d_repeat_num', type=int, default=8, help='number of strided conv layers in D') # was 6
    parser.add_argument('--lambda_cls', type=float, default=1, help='weight for domain classification loss')
    parser.add_argument('--lambda_rec', type=float, default=10, help='weight for reconstruction loss')
    parser.add_argument('--lambda_gp', type=float, default=10, help='weight for gradient penalty')
    parser.add_argument('--lambda_id', type=float, default=10, help='weight for identity loss')
    parser.add_argument('--depth', type=int, default=5, help='Number of output images/layers')

    # Training configuration.
    parser.add_argument('--dataset', type=str, default='CelebA', choices=['CelebA', 'BRATS', 'Directory'])#CelebA
    parser.add_argument('--batch_size', type=int, default=16, help='mini-batch size')
    parser.add_argument('--num_iters', type=int, default=200000, help='number of total iterations for training D')
    parser.add_argument('--num_iters_decay', type=int, default=100000, help='number of iterations for decaying lr')
    parser.add_argument('--g_lr', type=float, default=0.0001, help='learning rate for G')
    parser.add_argument('--d_lr', type=float, default=0.0001, help='learning rate for D')
    parser.add_argument('--n_critic', type=int, default=5, help='number of D updates per each G update')
    parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
    parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
    parser.add_argument('--resume_iters', type=int, default=None, help='resume training from this step')
    parser.add_argument('--selected_attrs', '--list', nargs='+', help='selected attributes for the CelebA dataset',
                        default=['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young'])

    # Test configuration.
    parser.add_argument('--test_iters', type=int, default=200000, help='test model from this step')

    # Miscellaneous.
    parser.add_argument('--num_workers', type=int, default=1)
    parser.add_argument('--mode', type=str, default='train', choices=['train', 'test', 'test_brats'])
    parser.add_argument('--use_tensorboard', type=str2bool, default=True)
    parser.add_argument('--job-dir', help=" will be passed through from the AI-Platform to your program when you define it.")
    #parser.add_argument('--ngpu', type=int, default=0)
    parser.add_argument('--GCP_project_id', type=str, default='fpcityuni-2020')
    parser.add_argument('--GCP_Bucket_name', type=str, default="fpcityuni-2020-storage")
    
    # Directories.
    parser.add_argument('--image_dir', type=str, default='data/celeba/images')
    
    parser.add_argument('--dataset_in_cloud', type=str, default='dataset.zip')
    parser.add_argument('--dataset_folder_name', type=str, default='data')
    parser.add_argument('--attr_path', type=str, default='data/celeba/list_attr_celeba.txt')
    parser.add_argument('--log_dir', type=str, default='celeba/logs')
    parser.add_argument('--model_save_dir', type=str, default='celeba/models')
    parser.add_argument('--sample_dir', type=str, default='celeba/samples')
    parser.add_argument('--result_dir', type=str, default='celeba/results')

    # Step size.
    parser.add_argument('--log_step', type=int, default=10)
    parser.add_argument('--sample_step', type=int, default=1000)
    parser.add_argument('--model_save_step', type=int, default=10000)
    parser.add_argument('--lr_update_step', type=int, default=1000)

    config = parser.parse_args()
    print(config, flush='True')
    print('__Tensorflow VERSION',tf.__version__)
    print('__Tensorboard VERSION:', version.VERSION)
    print('__pyTorch VERSION:', torch.__version__)
    #import torchvision
    #print('__Torchvision VERSION',torchvision.__version__)
    print('__CUDA VERSION',torch.version.cuda )
    from subprocess import call
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__Devices')
    if torch.cuda.device_count() >0:
      print('Active CUDA Device: GPU', torch.cuda.current_device())
      print('Current device name',torch.cuda.get_device_name(0))
    else:
      print('No Cuda Device')
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    torch.autograd.set_detect_anomaly(True)
    main(config)
    print(torch.cuda.memory_allocated())


Overwriting GAN_nGPU_GCP_Method2_scaledimagesaveraging_v2.py


# Method 2 Execution

In [9]:
! python3 -u GAN_nGPU_GCP_Method2_scaledimagesaveraging_v2.py --mode train --dataset 'CelebA' --image_size 1024 --crop_size 1024 --c_dim 5 --g_conv_dim 16 --d_conv_dim 16 --depth 4 --d_repeat_num 8 --g_repeat_num 6 \
                 --GCP_project_id fpcityuni-2020 \
                 --GCP_Bucket_name fpcityuni-2020-storage \
                 --image_dir data/test_max_size_of_readable_images \
                 --attr_path list_attr_celeba_hd.txt \
                 --sample_dir celeba_1/samples \
                 --log_dir celeba_1/logs \
                 --model_save_dir celeba_1/models \
                 --result_dir celeba_1/results \
                 --selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young --lambda_id 10 \
                 --batch_size 3 --num_workers 3 --num_iters 50000 --log_step 50 --sample_step 5000 #--resume_iters 10000

2020-08-15 21:34:32.754777: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Curret Directory:  /content/drive/My Drive/FinalProject-GCP
Namespace(GCP_Bucket_name='fpcityuni-2020-storage', GCP_project_id='fpcityuni-2020', attr_path='list_attr_celeba_hd.txt', batch_size=3, beta1=0.5, beta2=0.999, c2_dim=8, c_dim=5, crop_size=1024, d_conv_dim=16, d_lr=0.0001, d_repeat_num=8, dataset='CelebA', dataset_folder_name='data', dataset_in_cloud='dataset.zip', depth=4, g_conv_dim=16, g_lr=0.0001, g_repeat_num=6, image_dir='data/test_max_size_of_readable_images', image_size=1024, job_dir=None, lambda_cls=1, lambda_gp=10, lambda_id=10.0, lambda_rec=10, log_dir='celeba_1/logs', log_step=50, lr_update_step=1000, mode='train', model_save_dir='celeba_1/models', model_save_step=10000, n_critic=5, num_iters=50000, num_iters_decay=100000, num_workers=3, result_dir='celeba_1/results', resume_iters=None, sample_dir='celeba_1/samples', samp

# Testing for 1 to 1 *Translation*

In [ ]:
! python3 -u GAN_nGPU_GCP_Method2_scaledimagesaveraging_v2.py --mode train --dataset 'CelebA' --image_size 1024 --crop_size 1024 --c_dim 1 --selected_attrs Wearing_Hat \
                  --g_conv_dim 16 --d_conv_dim 16 --depth 4 --d_repeat_num 8 --g_repeat_num 6 \
                 --GCP_project_id fpcityuni-2020 \
                 --GCP_Bucket_name fpcityuni-2020-storage \
                 --image_dir data/test_max_size_of_readable_images \
                 --attr_path list_attr_celeba_hd.txt \
                 --sample_dir celeba_1/samples \
                 --log_dir celeba_1/logs \
                 --model_save_dir celeba_1/models \
                 --result_dir celeba_1/results \
                 --lambda_id 10 \
                 --batch_size 3 --num_workers 3 --num_iters 50000 --log_step 50 --sample_step 5000 #--resume_iters 10000 #--selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young 

2020-08-15 14:56:03.235303: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Curret Directory:  /content/drive/My Drive/FinalProject-GCP
Namespace(GCP_Bucket_name='fpcityuni-2020-storage', GCP_project_id='fpcityuni-2020', attr_path='list_attr_celeba_hd.txt', batch_size=3, beta1=0.5, beta2=0.999, c2_dim=8, c_dim=1, crop_size=1024, d_conv_dim=16, d_lr=0.0001, d_repeat_num=8, dataset='CelebA', dataset_folder_name='data', dataset_in_cloud='dataset.zip', depth=4, g_conv_dim=16, g_lr=0.0001, g_repeat_num=6, image_dir='data/test_max_size_of_readable_images', image_size=1024, job_dir=None, lambda_cls=1, lambda_gp=10, lambda_id=10.0, lambda_rec=10, log_dir='celeba_1/logs', log_step=50, lr_update_step=1000, mode='train', model_save_dir='celeba_1/models', model_save_step=10000, n_critic=5, num_iters=50000, num_iters_decay=100000, num_workers=3, result_dir='celeba_1/results', resume_iters=None, sample_dir='celeba_1/samples', samp